**HƯỚNG DẪN CHẠY**

*Nhóm chạy code theo thứ tự từng cell từ trên xuống xuống dưới*

**Một số điểm lưu ý:**

- *Thời gian chạy các block đa số lâu (Khoảng 10 phút riêng block 6 khoảng 25 phút)*

- *Các block 8,9,10,11 có lưu kết quả file csv* 

- **Các file csv kết quả nhóm có upload lên github**

**Tải các thư viện cần thiết** 

In [111]:
!pip install pandas numpy torch scikit-learn matplotlib
!pip install --extra-index-url https://fiinquant.github.io/fiinquantx/simple fiinquantx
!pip install --upgrade --extra-index-url https://fiinquant.github.io/fiinquantx/simple fiinquantx

Looking in indexes: https://pypi.org/simple, https://fiinquant.github.io/fiinquantx/simple
Looking in indexes: https://pypi.org/simple, https://fiinquant.github.io/fiinquantx/simple

  Attempting uninstall: fiinquantx

    Found existing installation: FiinQuantX 0.1.35

    Uninstalling FiinQuantX-0.1.35:

      Successfully uninstalled FiinQuantX-0.1.35

   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- ------------------- 1/2 [fiinquantx]
   -------------------- --------

**Block 1: tải dữ liệu lịch sử và realtime**

In [ ]:
# Block 1 — Login & Lấy dữ liệu tất cả HOSE/HNX/UPCOM
import pandas as pd
from FiinQuantX import FiinSession, BarDataUpdate
# --- Login ---
username = "DSTC_18@fiinquant.vn"
password = "Fiinquant0606"

client = FiinSession(
    username=username,
    password=password
).login()

# --- Lấy danh sách cổ phiếu từng sàn ---
tickers_hose  = list(client.TickerList(ticker="VNINDEX"))     # HOSE
print(f"Số mã HOSE: {len(tickers_hose)}")

# --- Lấy dữ liệu lịch sử toàn bộ ---
event_history = client.Fetch_Trading_Data(
    realtime=False,
    tickers=tickers_hose,
    fields=['open','high','low','close','volume','bu','sd','fs','fn'], 
    adjusted=True,
    by="1d",
    from_date="2020-01-01"   # Lấy dữ liệu từ 2021 tới nay
)

df_all = event_history.get_data()
print("History ban đầu:", df_all.head())

# --- Callback realtime ---
def onDataUpdate(data: BarDataUpdate):
    global df_all
    df_update = data.to_dataFrame()
    df_all = pd.concat([df_all, df_update])
    df_all = df_all.drop_duplicates()
    print("Realtime update:")
    print(df_update.head())

# --- Bật realtime nối tiếp dữ liệu ---
event_realtime = client.Fetch_Trading_Data(
    realtime=True,
    tickers=tickers_hose,
    fields=['open','high','low','close','volume','bu','sd','fs','fn'], 
    adjusted=True,
    by="1d",
    period=1,
    callback=onDataUpdate
)


**Block 2: lấy dữ liệu FA, lọc các mã không hợp lệ**

In [ ]:
# Block 2 — Lấy dữ liệu FA theo quý (VNINDEX) với retry + backoff, skip ticker không hợp lệ và lưu incremental

import os
import re
import time
import random
import pandas as pd
from vnstock import Finance

# --- Dùng tickers_hose từ Block 1 nếu có, nếu không lấy lại từ client ---
try:
    tickers_vnindex = tickers_hose
except NameError:
    tickers_vnindex = list(client.TickerList(ticker="VNINDEX"))

print(f"Số mã VNINDEX tổng: {len(tickers_vnindex)}")

# --- Output files ---
master_file = "vnindex_fa_quarterly_vnstock.csv"
per_ticker_dir = "vnindex_fa_by_ticker"
invalid_file = "vnindex_invalid_tickers.txt"
os.makedirs(per_ticker_dir, exist_ok=True)

# --- Retry / backoff parameters ---
MAX_RETRIES = 6
BASE_DELAY = 5
BACKOFF_BASE = 2.0
JITTER = 1.0
RATE_LIMIT_PATTERNS = [
    r"rate limit exceeded",
    r"you have sent too many requests",
    r"too many requests",
    r"rate limit",
    r"vci.*rate",
]
INVALID_TICKER_PATTERNS = [
    r"mã chứng khoán không hợp lệ",
    r"chỉ cổ phiếu mới có thông tin",
    r"không phải mã cổ phiếu",
    r"invalid symbol",
    r"symbol is invalid",
]

def parse_wait_seconds_from_msg(msg):
    if not msg:
        return None
    m = re.search(r"after\s+(\d+)\s*seconds?", msg, flags=re.IGNORECASE)
    if m:
        return int(m.group(1))
    m2 = re.search(r"after\s+(\d+)\s*s\b", msg, flags=re.IGNORECASE)
    if m2:
        return int(m2.group(1))
    m3 = re.search(r"(\d+)", msg)
    if m3:
        return int(m3.group(1))
    return None

# --- Resume: đọc danh sách tickers đã xử lý trong master file nếu có ---
processed = set()
if os.path.exists(master_file):
    try:
        df_exist = pd.read_csv(master_file, usecols=['ticker'])
        processed.update(df_exist['ticker'].astype(str).unique().tolist())
        print(f"Resume: đã phát hiện {len(processed)} ticker đã xử lý trong '{master_file}'.")
    except Exception:
        pass

# --- Resume: đọc danh sách ticker invalid đã lưu trước đó ---
invalid_set = set()
if os.path.exists(invalid_file):
    try:
        with open(invalid_file, "r", encoding="utf-8") as f:
            for line in f:
                s = line.strip()
                if s:
                    invalid_set.add(s)
        processed.update(invalid_set)  # coi invalid như đã xử lý để skip lần sau
        print(f"Resume: {len(invalid_set)} ticker đã được đánh dấu INVALID và sẽ bị bỏ qua.")
    except Exception:
        pass

# --- Thu thập dữ liệu ---
fa_list = []
successful = []
skipped_invalid = []
failed = []

for t in tickers_vnindex:
    if t in processed:
        print(f"Skip {t} (đã có trong processed/master/invalid).")
        continue

    attempt = 0
    success = False
    last_exception_msg = None

    while attempt < MAX_RETRIES and not success:
        attempt += 1
        try:
            finance = Finance(symbol=t, source='VCI')
            df = finance.ratio(period='quarterly')
            df = pd.DataFrame(df) if df is not None else pd.DataFrame()

            if df.empty:
                print(f"[{t}] ⚠️ Không có dữ liệu FA (vnstock). Bỏ qua (không lưu).")
                # không ghi master nếu trống; đánh dấu là đã thử (không add vào processed để có thể thử lại sau nếu muốn)
                success = True
                break

            # Đảm bảo có cột ticker
            if 'ticker' not in df.columns:
                df['ticker'] = t
            else:
                df['ticker'] = df['ticker'].fillna(t)

            # Nếu có ReportDate (meta) thì drop đi
            if 'ReportDate' in df.columns:
                df = df.drop(columns=['ReportDate'])

            # Coerce numeric cho các cột (nếu có thể)
            for c in df.columns:
                if c != 'ticker':
                    df[c] = pd.to_numeric(df[c], errors='ignore')

            # --- Lưu per-ticker CSV ---
            per_file = os.path.join(per_ticker_dir, f"{t}_fa.csv")
            df.to_csv(per_file, index=False, encoding='utf-8-sig')

            # --- Append vào master CSV ---
            write_header = not os.path.exists(master_file)
            df.to_csv(master_file, mode='a', header=write_header, index=False, encoding='utf-8-sig')

            print(f"[{t}] ✓ Lấy xong và lưu (rows={len(df)}).")
            fa_list.append(df)
            successful.append(t)
            processed.add(t)  # mark processed so resume will skip next time
            success = True

            # nhẹ nhàng sleep giữa requests
            time.sleep(BASE_DELAY + random.uniform(0, JITTER))

        except Exception as e:
            msg = str(e)
            last_exception_msg = msg
            lower_msg = msg.lower()

            # --- Nếu lỗi do ticker không hợp lệ -> bỏ luôn, không retry ---
            is_invalid = any(re.search(pat, lower_msg) for pat in INVALID_TICKER_PATTERNS)
            if is_invalid:
                print(f"[{t}] ❌ Mã không hợp lệ — BỎ QUA (không retry). Message: {msg}")
                skipped_invalid.append(t)
                processed.add(t)
                # lưu vào file invalid để lần sau không thử lại
                try:
                    with open(invalid_file, "a", encoding="utf-8") as f:
                        f.write(t + "\n")
                except Exception:
                    pass
                break

            # --- Nếu nghi ngờ rate-limit hoặc server báo wait time ---
            is_rate_limit = any(pat in lower_msg for pat in RATE_LIMIT_PATTERNS)
            parsed_wait = parse_wait_seconds_from_msg(msg)

            if is_rate_limit or parsed_wait:
                wait_sec = parsed_wait if parsed_wait and parsed_wait > 0 else (BACKOFF_BASE ** attempt)
                wait_sec = wait_sec + random.uniform(0, JITTER)
                print(f"[{t}] ⚠️ Rate limit detected (attempt {attempt}/{MAX_RETRIES}). "
                      f"Đợi {wait_sec:.1f}s rồi thử lại. Message: {msg}")
                time.sleep(wait_sec)
                continue
            else:
                backoff = (BACKOFF_BASE ** attempt) + random.uniform(0, JITTER)
                print(f"[{t}] ⚠️ Lỗi khi fetch (attempt {attempt}/{MAX_RETRIES}): {msg}. "
                      f"Đợi {backoff:.1f}s rồi thử lại.")
                time.sleep(backoff)
                continue

    # nếu vòng retry kết thúc mà không success và không thuộc invalid -> mark failed
    if not success and t not in skipped_invalid:
        print(f"[{t}] ❌ Không lấy được dữ liệu sau {MAX_RETRIES} lần. Bỏ qua ticker này. Lỗi cuối: {last_exception_msg}")
        failed.append(t)

# --- Kết quả tổng kết ---
print("=== Hoàn tất Block 2 (vnstock) ===")
print(f"Tổng tickers xử lý thành công: {len(successful)}")
print(f"Tổng tickers bị đánh dấu INVALID và bỏ qua: {len(skipped_invalid)}")
print(f"Tổng tickers thất bại (retry hết nhưng không invalid): {len(failed)}")

if fa_list:
    sample_df = pd.concat(fa_list, ignore_index=True, sort=False).head(10)
    display(sample_df)
else:
    print("❗ Không có dữ liệu FA thu được cho các ticker đã chạy.")


Số mã VNINDEX tổng: 413
Resume: đã phát hiện 13756 ticker đã xử lý trong 'vnindex_fa_quarterly_vnstock.csv'.
Resume: 22 ticker đã được đánh dấu INVALID và sẽ bị bỏ qua.
Skip CCC (đã có trong processed/master/invalid).
Skip SBG (đã có trong processed/master/invalid).
Skip FUCTVGF3 (đã có trong processed/master/invalid).
Skip FUEIP100 (đã có trong processed/master/invalid).
[GMH] ✓ Lấy xong và lưu (rows=18).
Skip FUEKIV30 (đã có trong processed/master/invalid).
Skip NO1 (đã có trong processed/master/invalid).
Skip FUCTVGF4 (đã có trong processed/master/invalid).
[RYG] ✓ Lấy xong và lưu (rows=7).
Skip FUEDCMID (đã có trong processed/master/invalid).
Skip FUEKIVFS (đã có trong processed/master/invalid).
Skip FUEMAVND (đã có trong processed/master/invalid).
Skip FUEFCV50 (đã có trong processed/master/invalid).
Skip FUEBFVND (đã có trong processed/master/invalid).
Skip FUCTVGF5 (đã có trong processed/master/invalid).
Skip FUEKIVND (đã có trong processed/master/invalid).
Skip FUEABVND (đã có 

Meta                         Chỉ tiêu cơ cấu nguồn vốn              \
  ticker yearReport lengthReport (ST+LT borrowings)/Equity Debt/Equity   
0    GMH       2025            2                  0.000000    0.051588   
1    GMH       2025            1                  0.022473    0.051838   
2    GMH       2024            4                  0.000000    0.038005   
3    GMH       2024            3                  0.038418    0.106987   
4    GMH       2024            2                  0.015499    0.063503   
5    GMH       2024            1                  0.009567    0.032987   
6    GMH       2023            4                  0.000000    0.047413   
7    GMH       2023            3                  0.040551    0.115587   
8    GMH       2023            2                  0.000000    0.088011   
9    GMH       2023            1                  0.043288    0.101775   

                                                        \
  Fixed Asset-To-Equity Owners' Equity/Charter Capital   
0              0.124650                       1.090538   
1              0.130708                       1.089856   
2              0.138373                       1.077694   
3              0.146877                       1.065434   
4              0.156407                       1.048888   
5              0.150374                       1.142198   
6              0.157043                       1.142922   
7              0.165952                       1.128155   
8              0.175449                       1.105311   
9              0.169373                       1.137871   

  Chỉ tiêu hiệu quả hoạt động                                              \
               Asset Turnover Fixed Asset Turnover Days Sales Outstanding   
0                    0.562538             4.422492              63.351078   
1                    0.495824             3.693597              71.229634   
2                    0.461186             3.248087              69.708499   
3                    0.456444             3.178684              66.693418   
4                    0.438959             2.972644              64.985075   
5                    0.473768             3.133843              60.882722   
6                    0.572311             3.624814              53.195105   
7                    0.637397             4.058792              50.114334   
8                    0.682272             4.248544              47.441126   
9                    0.724659             4.420403              40.665490   

   ...        Chỉ tiêu định giá                                             \
   ... Market Capital (Bn. VND) Outstanding Share (Mil. Shares)        P/E   
0  ...             1.452000e+11                      16500000.0  11.480752   
1  ...             1.311750e+11                      16500000.0  16.681350   
2  ...             1.229250e+11                      16500000.0  20.639945   
3  ...             1.313400e+11                      16500000.0  21.487381   
4  ...             1.366200e+11                      16500000.0  19.196756   
5  ...             1.485000e+11                      16500000.0  16.702805   
6  ...             1.658250e+11                      16500000.0  12.102004   
7  ...             1.600500e+11                      16500000.0   9.127335   
8  ...             1.592250e+11                      16500000.0   8.101948   
9  ...             1.666500e+11                      16500000.0   6.932984   

                                                                       ticker  
        P/B       P/S P/Cash Flow   EPS (VND)    BVPS (VND)  EV/EBITDA         
0  0.806941  1.362896    3.795689  356.816813  10905.381178  11.204889    GMH  
1  0.729454  1.406508    3.940808  121.619694  10898.564366  20.561323    GMH  
2  0.691291  1.395969   10.732768  120.501797  10776.944672  19.378872    GMH  
3  0.747113  1.491061    7.664694  167.561987  10654.338632  31.526449    GMH  
4  0.789407  1.585955   11.360902   66.896642  10488.880887  18.004344    GM

**Block 3: Chuẩn hóa FA và gộp dữ liệu với giá**

In [ ]:
# Block 3 (fixed) — Chuẩn hoá FA (từ file per-ticker có 2 dòng header) + Merge với giá + Split train/val/test
import os
import pandas as pd
import numpy as np

# ---------- Config (chỉnh nếu cần) ----------
fa_folder = "vnindex_fa_by_ticker"
output_merged = "vnindex_price_fa_merged.csv"
output_train = "vnindex_price_fa_train.parquet"
output_val   = "vnindex_price_fa_val.parquet"
output_test  = "vnindex_price_fa_test.parquet"

# Default splits (thay đổi nếu muốn)
TRAIN_END = pd.Timestamp("2023-12-31")   # bao gồm ngày này -> TRAIN
VAL_END   = pd.Timestamp("2024-12-31")   # ngày > TRAIN_END & <= VAL_END -> VAL
# TEST = ngày > VAL_END

# Các cột FA bạn cần (theo yêu cầu)
fa_fields = ["Debt/Equity", "Net Profit Margin (%)", "P/E", "P/B"]
need_cols = ["ticker","yearReport","lengthReport"] + fa_fields

# ---------- 1) Đọc FA từ thư mục ----------
fa_list = []
if not os.path.isdir(fa_folder):
    raise FileNotFoundError(f"FA folder not found: {fa_folder}")

for file in os.listdir(fa_folder):
    if not file.endswith("_fa.csv"):
        continue
    path = os.path.join(fa_folder, file)
    try:
        # Bỏ dòng meta đầu, dòng thứ 2 làm header
        df = pd.read_csv(path, header=1, encoding="utf-8", low_memory=False)

        # Đảm bảo có tất cả các cột
        for col in need_cols:
            if col not in df.columns:
                df[col] = pd.NA

        df = df[need_cols].copy()
        # Chuẩn hoá ticker (loại khoảng trắng)
        df["ticker"] = df["ticker"].astype(str).str.strip().str.upper()
        fa_list.append(df)
    except Exception as e:
        print(f"⚠️ Lỗi đọc {file}: {e}")

if len(fa_list) == 0:
    raise RuntimeError("Không đọc được file FA nào từ folder. Kiểm tra đường dẫn và tên file *_fa.csv")

fa_data = pd.concat(fa_list, ignore_index=True)
print(f"Đã đọc FA từ {len(fa_list)} file, tổng dòng = {len(fa_data)}")

# ---------- 2) Load price df_all nếu cần ----------
if "df_all" not in globals():
    # cố gắng load từ file tiêu chuẩn nếu tồn tại
    possible_paths = ["vnindex_price_all.csv", "vnindex_price.csv", "vnindex_price_fa_merged.csv"]
    loaded = False
    for p in possible_paths:
        if os.path.exists(p):
            print(f"⚡ Không tìm thấy df_all trong memory, đang load từ: {p}")
            df_all = pd.read_csv(p, parse_dates=["timestamp"])
            loaded = True
            break
    if not loaded:
        raise RuntimeError("Không tìm thấy biến df_all trong bộ nhớ và không tìm được file giá mặc định. Vui lòng chạy Block 1 hoặc đặt file price tại one of: " + ", ".join(possible_paths))

# Chuẩn hoá df_all minimal
df_all = df_all.rename(columns={c:c for c in df_all.columns})
if "timestamp" not in df_all.columns:
    raise RuntimeError("df_all phải có cột 'timestamp' (datetime).")

df_all["timestamp"] = pd.to_datetime(df_all["timestamp"])
df_all["ticker"] = df_all["ticker"].astype(str).str.strip().str.upper()
df_all = df_all.sort_values(["ticker","timestamp"]).reset_index(drop=True)

# Giới hạn df_price chỉ bao những ticker có FA (đỡ nặng)
tickers_with_fa = fa_data["ticker"].dropna().unique().tolist()
df_price = df_all[df_all["ticker"].isin(tickers_with_fa)].copy()
print("Số bản ghi giá (sau filter theo FA):", len(df_price), " | Số ticker:", df_price["ticker"].nunique())

# ---------- 3) Tạo key fa_year/fa_quarter = quý TRƯỚC của price timestamp ----------
# (Giả định: FA áp dụng lag 1 quarter)
pi = df_price["timestamp"].dt.to_period("Q")
prev_pi = pi - 1
df_price["fa_year"] = prev_pi.dt.year.astype(int)
df_price["fa_quarter"] = prev_pi.dt.quarter.astype(int)

# ---------- 4) Chuẩn hoá bảng FA (rename + dedup) ----------
fa_clean = fa_data.rename(columns={"yearReport": "fa_year", "lengthReport": "fa_quarter"}).copy()

# ép kiểu numeric nơi có thể (giữ NA nếu không parse được)
for f in fa_fields:
    if f in fa_clean.columns:
        fa_clean[f] = pd.to_numeric(fa_clean[f], errors="coerce")

# đảm bảo ticker, fa_year, fa_quarter có
fa_clean["ticker"] = fa_clean["ticker"].astype(str).str.strip().str.upper()
# cast year/quarter sang Int64 (nullable) nếu có
if "fa_year" in fa_clean.columns:
    fa_clean["fa_year"] = pd.to_numeric(fa_clean["fa_year"], errors="coerce").astype("Int64")
if "fa_quarter" in fa_clean.columns:
    fa_clean["fa_quarter"] = pd.to_numeric(fa_clean["fa_quarter"], errors="coerce").astype("Int64")

# drop duplicate (giữ bản cuối cùng theo sort)
fa_clean = fa_clean.sort_values(["ticker","fa_year","fa_quarter"]).drop_duplicates(subset=["ticker","fa_year","fa_quarter"], keep="last")

print("FA cleaned: tickers:", fa_clean["ticker"].nunique(), "rows:", len(fa_clean))

# ---------- 5) Merge price + FA on (ticker, fa_year, fa_quarter) ----------
df_merged = df_price.merge(
    fa_clean,
    on=["ticker","fa_year","fa_quarter"],
    how="left",
    sort=True,
    suffixes=("","_fa")
)

# ---------- 6) Forward-fill FA theo ticker (chỉ sử dụng thông tin quá khứ) ----------
# Lưu ý: ffill groupby theo timestamp tăng dần đảm bảo không leak (ffill chỉ dùng giá trị trước đó)
df_merged = df_merged.sort_values(["ticker","timestamp"]).reset_index(drop=True)
# Cast fa fields numeric to avoid object issues
for f in fa_fields:
    if f not in df_merged.columns:
        df_merged[f] = np.nan
    else:
        df_merged[f] = pd.to_numeric(df_merged[f], errors="coerce")

# ffill per ticker
df_merged[fa_fields] = df_merged.groupby("ticker")[fa_fields].ffill()

# ---------- 7) Sanity checks ----------
print("Merged shape:", df_merged.shape)
print("Merged time range:", df_merged["timestamp"].min(), "->", df_merged["timestamp"].max())
print("Ticker count:", df_merged["ticker"].nunique())

# Báo cáo số row không có FA (sau ffill)
no_fa_rows = df_merged[fa_fields].isna().all(axis=1).sum()
print(f"Số dòng vẫn thiếu toàn bộ FA sau ffill: {no_fa_rows} (có thể là ticker hoàn toàn không có báo cáo trước thời điểm đó)")

# ---------- 😎 Lưu merged CSV (backup) ----------
df_merged.to_csv(output_merged, index=False, encoding="utf-8-sig")
print(f"✅ Đã lưu merged -> {output_merged}")

# ---------- 9) Split train / val / test theo timestamp (lưu parquet để dùng tiếp) ----------
df_merged["timestamp"] = pd.to_datetime(df_merged["timestamp"])

df_train = df_merged[df_merged["timestamp"] <= TRAIN_END].copy()
df_val   = df_merged[(df_merged["timestamp"] > TRAIN_END) & (df_merged["timestamp"] <= VAL_END)].copy()
df_test  = df_merged[df_merged["timestamp"] > VAL_END].copy()

print("Split sizes (rows): TRAIN", len(df_train), "VAL", len(df_val), "TEST", len(df_test))
print("TRAIN period:", df_train["timestamp"].min(), "->", df_train["timestamp"].max() if len(df_train)>0 else "N/A")
print("VAL period:", df_val["timestamp"].min() if len(df_val)>0 else "N/A", "->", df_val["timestamp"].max() if len(df_val)>0 else "N/A")
print("TEST period:", df_test["timestamp"].min() if len(df_test)>0 else "N/A", "->", df_test["timestamp"].max() if len(df_test)>0 else "N/A")

# Save splits (parquet preserves dtypes)
df_train.to_parquet(output_train, index=False)
df_val.to_parquet(output_val, index=False)
df_test.to_parquet(output_test, index=False)
print(f"✅ Saved splits -> {output_train}, {output_val}, {output_test}")

# ---------- 10) Quick summary per ticker (optional) ----------
summary = df_merged.groupby("ticker").agg({
    "timestamp": ["min","max","count"]
})
summary.columns = ["first_date","last_date","n_rows"]
summary = summary.reset_index().sort_values("n_rows", ascending=False)
print("Top tickers by data rows:")
print(summary.head(8))

# ---------- End ----------

Đã đọc FA từ 391 file, tổng dòng = 17586
Số bản ghi giá (sau filter theo FA): 546084  | Số ticker: 391
FA cleaned: tickers: 391 rows: 17586
Merged shape: (546084, 17)
Merged time range: 2020-01-02 00:00:00 -> 2025-10-02 00:00:00
Ticker count: 391
Số dòng vẫn thiếu toàn bộ FA sau ffill: 245 (có thể là ticker hoàn toàn không có báo cáo trước thời điểm đó)
✅ Đã lưu merged -> vnindex_price_fa_merged.csv
Split sizes (rows): TRAIN 376939 VAL 96924 TEST 72221
TRAIN period: 2020-01-02 00:00:00 -> 2023-12-29 00:00:00
VAL period: 2024-01-02 00:00:00 -> 2024-12-31 00:00:00
TEST period: 2025-01-02 00:00:00 -> 2025-10-02 00:00:00
✅ Saved splits -> vnindex_price_fa_train.parquet, vnindex_price_fa_val.parquet, vnindex_price_fa_test.parquet
Top tickers by data rows:
    ticker first_date  last_date  n_rows
390    YEG 2020-01-02 2025-10-02    1435
0      AAA 2020-01-02 2025-10-02    1435
1      AAM 2020-01-02 2025-10-02    1435
389    YBM 2020-01-02 2025-10-02    1435
371    VNM 2020-01-02 2025-10-02  

**Xóa biến df_all không cần thiết nữa để giảm dung lượng RAM**

**Block 4: Tính các chỉ số TA dựa vào thư viện FiinQuant và ghép dữ liệu**

In [ ]:
# Block 4 — Tính các chỉ số TA cho từng mã (split cố định: Train 2020-2023, Val 2024, Test 2025->)
import pandas as pd
import numpy as np
from datetime import datetime

# ---------- Config (đã chốt) ----------
MERGED_FILE = "vnindex_price_fa_merged.csv"   # file output từ Block 3
TRAIN_START = pd.Timestamp("2020-01-01")
TRAIN_END   = pd.Timestamp("2023-12-31")
VAL_START   = pd.Timestamp("2024-01-01")
VAL_END     = pd.Timestamp("2024-12-31")
TEST_START  = pd.Timestamp("2025-01-01")
# TEST_END = inferred from data

OUT_TRAIN = "vnindex_price_fa_ta_train.csv"
OUT_VAL   = "vnindex_price_fa_ta_val.csv"
OUT_TEST  = "vnindex_price_fa_ta_test.csv"

# ---------- Load merged ----------
print("Loading merged file:", MERGED_FILE)
df = pd.read_csv(MERGED_FILE, parse_dates=True)

# support both 'timestamp' or 'date' column names
if "timestamp" in df.columns:
    df.rename(columns={"timestamp":"date"}, inplace=True)
if "date" not in df.columns:
    raise ValueError("Input merged file must contain a 'timestamp' or 'date' column.")

df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["ticker","date"]).reset_index(drop=True)

# infer TEST_END as last date in data
TEST_END = df["date"].max()
print(f"Data range: {df['date'].min().date()} → {df['date'].max().date()}")
print("Using splits:")
print(" TRAIN:", TRAIN_START.date(), "→", TRAIN_END.date())
print(" VAL  :", VAL_START.date(), "→", VAL_END.date())
print(" TEST :", TEST_START.date(), "→", TEST_END.date())

# ---------- TA helper: try client.FiinIndicator else fallback ----------
use_fi = False
try:
    fi = client.FiinIndicator()
    if hasattr(fi, "ema"):
        use_fi = True
        print("Using client.FiinIndicator() for TA.")
except Exception:
    use_fi = False
    print("FiIndicator not available — using pandas fallback implementations for TA.")

# fallback TA implementations
def ema(series, window):
    return series.ewm(span=window, adjust=False).mean()

def macd_series(close, fast=12, slow=26, signal=9):
    ema_fast = ema(close, fast)
    ema_slow = ema(close, slow)
    macd = ema_fast - ema_slow
    macd_signal = ema(macd, signal)
    macd_diff = macd - macd_signal
    return macd, macd_signal, macd_diff

def rsi(series, window=14):
    delta = series.diff()
    up = delta.clip(lower=0.0)
    down = -delta.clip(upper=0.0)
    ma_up = up.rolling(window).mean()
    ma_down = down.rolling(window).mean()
    rs = ma_up / (ma_down.replace(0, np.nan))
    rsi_v = 100 - (100 / (1 + rs))
    return rsi_v.fillna(0.0)

def bollinger_bands(close, window=20, dev=2):
    m = close.rolling(window).mean()
    s = close.rolling(window).std()
    return m + dev*s, m - dev*s

def atr(high, low, close, window=14):
    prev_close = close.shift(1)
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window).mean()

def obv(close, volume):
    sign = np.sign(close.diff().fillna(0.0))
    return (sign * volume.fillna(0.0)).cumsum()

def vwap(high, low, close, volume, window=14):
    tp = (high + low + close) / 3.0
    pv = tp * volume
    vp = pv.rolling(window).sum()
    vv = volume.rolling(window).sum()
    return (vp / vv).fillna(np.nan)

# ---------- function to compute TA per ticker ----------
def add_ta_indicators(df_t):
    df_t = df_t.sort_values("date").copy().reset_index(drop=True)
    close = df_t["close"].astype(float)

    # Prefer fi if available
    if use_fi:
        try:
            df_t['ema_5']  = fi.ema(df_t['close'], window=5)
            df_t['ema_20'] = fi.ema(df_t['close'], window=20)
            df_t['ema_50'] = fi.ema(df_t['close'], window=50)
            df_t['macd']        = fi.macd(df_t['close'], window_fast=12, window_slow=26)
            df_t['macd_signal'] = fi.macd_signal(df_t['close'], window_fast=12, window_slow=26, window_sign=9)
            df_t['macd_diff']   = fi.macd_diff(df_t['close'], window_fast=12, window_slow=26, window_sign=9)
            df_t['rsi'] = fi.rsi(df_t['close'], window=14)
            df_t['bollinger_hband'] = fi.bollinger_hband(df_t['close'], window=20, window_dev=2)
            df_t['bollinger_lband'] = fi.bollinger_lband(df_t['close'], window=20, window_dev=2)
            df_t['atr'] = fi.atr(df_t['high'], df_t['low'], df_t['close'], window=14)
            df_t['obv'] = fi.obv(df_t['close'], df_t['volume'])
            df_t['vwap'] = fi.vwap(df_t['high'], df_t['low'], df_t['close'], df_t['volume'], window=14)
            return df_t
        except Exception:
            # fallback to pandas implementations if fi call fails for any ticker
            pass

    # fallback calculations
    df_t['ema_5']  = ema(close, 5)
    df_t['ema_20'] = ema(close, 20)
    df_t['ema_50'] = ema(close, 50)
    macd, macd_signal, macd_diff = macd_series(close, 12, 26, 9)
    df_t['macd'] = macd
    df_t['macd_signal'] = macd_signal
    df_t['macd_diff'] = macd_diff
    df_t['rsi'] = rsi(close, 14)
    bh, bl = bollinger_bands(close, 20, 2)
    df_t['bollinger_hband'] = bh
    df_t['bollinger_lband'] = bl

    try:
        df_t['atr'] = atr(df_t['high'].astype(float), df_t['low'].astype(float), close, window=14)
    except Exception:
        df_t['atr'] = np.nan

    try:
        df_t['obv'] = obv(close, df_t['volume'].fillna(0.0).astype(float))
    except Exception:
        df_t['obv'] = np.nan

    try:
        df_t['vwap'] = vwap(df_t['high'].astype(float), df_t['low'].astype(float), close, df_t['volume'].fillna(0.0).astype(float), window=14)
    except Exception:
        df_t['vwap'] = np.nan

    return df_t

# ---------- process function and save ----------
def process_and_save(df_slice, name, out_path):
    print(f"Processing {name}: rows={df_slice.shape[0]} | tickers={df_slice['ticker'].nunique()}")
    res = df_slice.groupby("ticker", group_keys=False).apply(add_ta_indicators)
    res = res.sort_values(["ticker","date"]).reset_index(drop=True)
    res.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"Saved {name} -> {out_path} | shape: {res.shape}")
    return res

# ---------- create splits ----------
df_train = df[(df["date"] >= TRAIN_START) & (df["date"] <= TRAIN_END)].copy()
df_val   = df[(df["date"] >= VAL_START)   & (df["date"] <= VAL_END)].copy()
df_test  = df[(df["date"] >= TEST_START)  & (df["date"] <= TEST_END)].copy()

# safety warnings
if df_train.empty:
    print("⚠️ TRAIN split empty — check data and TRAIN_START/END.")
if df_val.empty:
    print("⚠️ VAL split empty — check VAL_START/END.")
if df_test.empty:
    print("⚠️ TEST split empty — check TEST_START / data availability (need 2025+).")

# run
df_train_ta = process_and_save(df_train, "TRAIN (2020-01-01 → 2023-12-31)", OUT_TRAIN) if not df_train.empty else pd.DataFrame()
df_val_ta   = process_and_save(df_val,   "VAL   (2024)", OUT_VAL)   if not df_val.empty else pd.DataFrame()
df_test_ta  = process_and_save(df_test,  "TEST  (2025→)", OUT_TEST)  if not df_test.empty else pd.DataFrame()

print("✅ Block 4 complete. Generated files:")
print("  -", OUT_TRAIN)
print("  -", OUT_VAL)
print("  -", OUT_TEST)

Loading merged file: vnindex_price_fa_merged.csv
Data range: 2020-01-02 → 2025-10-02
Using splits:
 TRAIN: 2020-01-01 → 2023-12-31
 VAL  : 2024-01-01 → 2024-12-31
 TEST : 2025-01-01 → 2025-10-02
Using client.FiinIndicator() for TA.
Processing TRAIN (2020-01-01 → 2023-12-31): rows=376939 | tickers=385
Saved TRAIN (2020-01-01 → 2023-12-31) -> vnindex_price_fa_ta_train.csv | shape: (376939, 29)
Processing VAL   (2024): rows=96924 | tickers=390
Saved VAL   (2024) -> vnindex_price_fa_ta_val.csv | shape: (96924, 29)
Processing TEST  (2025→): rows=72221 | tickers=391
Saved TEST  (2025→) -> vnindex_price_fa_ta_test.csv | shape: (72221, 29)
✅ Block 4 complete. Generated files:
  - vnindex_price_fa_ta_train.csv
  - vnindex_price_fa_ta_val.csv
  - vnindex_price_fa_ta_test.csv


**Xóa bớt biến df_merged không còn cần thiết để giảm dung lượng RAM**

In [ ]:
import gc
del df_merged
gc.collect()

1802

**Block 5: Chuẩn hóa dữ liệu FA và TA**

In [ ]:
# Block 5 — Feature engineering & scaling (NO regime, compatible with splits)
import os
import numpy as np
import pandas as pd
from datetime import datetime

# ---------- Config ----------
# Feature lists (the ones you specified)
fa_features = ["Debt/Equity", "Net Profit Margin (%)", "P/E", "P/B"]

ta_features = [
    "ema_5","ema_20","ema_50","macd","macd_signal","macd_diff",
    "rsi","bollinger_hband","bollinger_lband","atr","obv","vwap"
]

ROLL_WINDOW = 60   # rolling window for TA z-score (days)
MIN_ROWS_KEEP = 1  # minimum rows per ticker to compute rolling z (adjust if needed)

# File paths produced by Block 4
FULL_TA_FILE = "vnindex_price_fa_ta.csv"   # if exists, use it (preferred)
TRAIN_TA_FILE = "vnindex_price_fa_ta_train.csv"
VAL_TA_FILE   = "vnindex_price_fa_ta_val.csv"
TEST_TA_FILE  = "vnindex_price_fa_ta_test.csv"

OUT_DIR = "./features/"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_TRAIN = os.path.join(OUT_DIR, "df_features_train.csv")
OUT_VAL   = os.path.join(OUT_DIR, "df_features_val.csv")
OUT_TEST  = os.path.join(OUT_DIR, "df_features_test.csv")

# Date splits (you confirmed these)
TRAIN_START = pd.Timestamp("2020-01-01"); TRAIN_END = pd.Timestamp("2023-12-31")
VAL_START   = pd.Timestamp("2024-01-01"); VAL_END   = pd.Timestamp("2024-12-31")
TEST_START  = pd.Timestamp("2025-01-01")  # TEST_END inferred from data

# ---------- Load TA data (prefer full timeline to compute rolling correctly) ----------
if os.path.exists(FULL_TA_FILE):
    print("Loading full TA file:", FULL_TA_FILE)
    df_ta = pd.read_csv(FULL_TA_FILE, parse_dates=["date"] if "date" in pd.read_csv(FULL_TA_FILE, nrows=0).columns else ["timestamp"])
    # normalize column name to 'date'
    if "timestamp" in df_ta.columns and "date" not in df_ta.columns:
        df_ta = df_ta.rename(columns={"timestamp":"date"})
else:
    # try to load the three split files and concat (they were created earlier in Block 4)
    parts = []
    for p in [TRAIN_TA_FILE, VAL_TA_FILE, TEST_TA_FILE]:
        if os.path.exists(p):
            tmp = pd.read_csv(p, parse_dates=["date"] if "date" in pd.read_csv(p, nrows=0).columns else ["timestamp"])
            if "timestamp" in tmp.columns and "date" not in tmp.columns:
                tmp = tmp.rename(columns={"timestamp":"date"})
            parts.append(tmp)
    if not parts:
        raise FileNotFoundError("Cannot find TA input files. Provide vnindex_price_fa_ta.csv or the 3 split files from Block 4.")
    df_ta = pd.concat(parts, ignore_index=True).sort_values(["ticker","date"]).reset_index(drop=True)
    print("Concatenated TA splits; shape:", df_ta.shape)

# ensure date col and sorted
if "date" not in df_ta.columns:
    raise ValueError("Input TA dataframe must contain 'date' or 'timestamp' column (normalized to 'date').")
df_ta["date"] = pd.to_datetime(df_ta["date"])
df_ta = df_ta.sort_values(["ticker","date"]).reset_index(drop=True)
TEST_END = df_ta["date"].max()
print("Data timeline:", df_ta["date"].min().date(), "→", TEST_END.date())

# ---------- 1) FA cross-sectional min-max per day ----------
# We'll compute per-day min/max across tickers for each FA column, then scale per day.
def scale_fa_minmax_group(df_day):
    # df_day is all tickers for a single date
    out = df_day.copy()
    for f in fa_features:
        vals = pd.to_numeric(df_day.get(f, pd.Series(dtype=float)), errors="coerce")
        vmin = vals.min()
        vmax = vals.max()
        if np.isfinite(vmin) and np.isfinite(vmax) and (vmax > vmin):
            out[f] = (vals - vmin) / (vmax - vmin)
        else:
            out[f] = np.nan
    return out

print("Computing FA cross-sectional min-max per date...")
# apply per date
df_fa_scaled = df_ta.groupby("date", group_keys=False).apply(scale_fa_minmax_group)
df_fa_scaled = df_fa_scaled.reset_index(drop=True)

# ---------- 2) TA rolling z-score per ticker (computed on full timeline -> safe) ----------
print(f"Computing rolling z-score (window={ROLL_WINDOW}) per ticker for TA features...")
def compute_zscores_for_ticker(g):
    g = g.sort_values("date").copy()
    for col in ta_features:
        if col not in g.columns:
            g[col] = np.nan
        # compute rolling mean/std and z-score (use center=False so it's causal)
        roll_mean = g[col].rolling(window=ROLL_WINDOW, min_periods=1).mean()
        roll_std  = g[col].rolling(window=ROLL_WINDOW, min_periods=1).std(ddof=0).replace(0, np.nan)
        g[f"{col}_z"] = (g[col] - roll_mean) / roll_std
    return g

df_all = df_fa_scaled.groupby("ticker", group_keys=False).apply(compute_zscores_for_ticker).reset_index(drop=True)

# ---------- 3) Keep only required columns and drop rows without features ----------
keep_cols = ["ticker","date"] + fa_features + [f"{col}_z" for col in ta_features]
# ensure exist
for c in keep_cols:
    if c not in df_all.columns:
        # if missing, create as NaN (to keep schema)
        df_all[c] = np.nan

df_features = df_all[keep_cols].copy()

# Drop rows with NaN in all features (or you can be stricter: drop if any feature NaN)
# Here: require FA and at least one TA_z not-null
ta_z_cols = [f"{col}_z" for col in ta_features]
mask_valid = df_features[fa_features].notna().any(axis=1) & df_features[ta_z_cols].notna().any(axis=1)
df_features = df_features[mask_valid].reset_index(drop=True)

print("After feature construction, rows:", len(df_features), "tickers:", df_features['ticker'].nunique())

# ---------- 4) Split back to TRAIN / VAL / TEST and save ----------
df_train_feats = df_features[(df_features["date"] >= TRAIN_START) & (df_features["date"] <= TRAIN_END)].copy()
df_val_feats   = df_features[(df_features["date"] >= VAL_START)   & (df_features["date"] <= VAL_END)].copy()
df_test_feats  = df_features[(df_features["date"] >= TEST_START)  & (df_features["date"] <= TEST_END)].copy()

print("Split sizes (train/val/test):", df_train_feats.shape, df_val_feats.shape, df_test_feats.shape)

# Save
df_train_feats.to_csv(OUT_TRAIN, index=False, encoding="utf-8-sig")
df_val_feats.to_csv(OUT_VAL, index=False, encoding="utf-8-sig")
df_test_feats.to_csv(OUT_TEST, index=False, encoding="utf-8-sig")

print("Saved features to:")
print("  -", OUT_TRAIN)
print("  -", OUT_VAL)
print("  -", OUT_TEST)

Loading full TA file: vnindex_price_fa_ta.csv
Data timeline: 2023-01-03 → 2025-10-02
Computing FA cross-sectional min-max per date...
Computing rolling z-score (window=60) per ticker for TA features...
After feature construction, rows: 264103 tickers: 391
Split sizes (train/val/test): (95000, 18) (96915, 18) (72188, 18)
Saved features to:
  - ./features/df_features_train.csv
  - ./features/df_features_val.csv
  - ./features/df_features_test.csv


**Block 6: Giảm chiều dữ liệu bằng t-SNE và phân cụm bằng DBSCAN**

In [ ]:
# Block 6 — Clustering safe: K = 10 on TRAIN + assign to VAL/TEST (fixed)
import os, gc, joblib
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# ---------- CONFIG ----------
TRAIN_FILE = "./features/df_features_train.csv"
VAL_FILE   = "./features/df_features_val.csv"
TEST_FILE  = "./features/df_features_test.csv"

OUT_DIR = "./clusters/"
os.makedirs(OUT_DIR, exist_ok=True)

RANDOM_STATE = 42
K_FIXED = 10          # <- cố định thành 10 theo yêu cầu
IMPUTE_STRAT = "median"
PCA_DIM = None        # hoặc số nguyên (ví dụ 20) nếu muốn giảm chiều trước khi cluster
VERBOSE = True
# -------------------------------

# --- Load splits (Block5 outputs) ---
df_train = pd.read_csv(TRAIN_FILE)
df_val   = pd.read_csv(VAL_FILE)
df_test  = pd.read_csv(TEST_FILE)

# --- Normalize date column names (accept "date" or "timestamp") ---
def ensure_date_col(df):
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
    elif "timestamp" in df.columns:
        df["date"] = pd.to_datetime(df["timestamp"])
    else:
        raise ValueError("Input file must contain 'date' or 'timestamp' column")
    return df

df_train = ensure_date_col(df_train)
df_val   = ensure_date_col(df_val)
df_test  = ensure_date_col(df_test)

# --- Feature columns inference (must match Block5) ---
fa_features = ["Debt/Equity", "Net Profit Margin (%)", "P/E", "P/B"]
ta_z_cols = [c for c in df_train.columns if c.endswith("_z")]
feature_cols = [c for c in (fa_features + ta_z_cols) if c in df_train.columns]

if len(feature_cols) == 0:
    raise ValueError("No feature columns found. Check Block5 output and feature names.")

if VERBOSE:
    print("Features used for clustering:", feature_cols)
    print("Train/Val/Test shapes:", df_train.shape, df_val.shape, df_test.shape)

# --- Build raw matrices ---
X_train_raw = df_train[feature_cols].values
X_val_raw   = df_val[feature_cols].values
X_test_raw  = df_test[feature_cols].values
X_all_raw   = np.vstack([X_train_raw, X_val_raw, X_test_raw])

# --- Imputer: fit on TRAIN only ---
imp = SimpleImputer(strategy=IMPUTE_STRAT)
imp.fit(X_train_raw)             # fit only on train
X_train_imp = imp.transform(X_train_raw)
X_val_imp   = imp.transform(X_val_raw)
X_test_imp  = imp.transform(X_test_raw)
X_all_imp   = imp.transform(X_all_raw)

# --- Scaler: fit on TRAIN only ---
scaler = StandardScaler()
scaler.fit(X_train_imp)
X_train_scaled = scaler.transform(X_train_imp)
X_val_scaled   = scaler.transform(X_val_imp)
X_test_scaled  = scaler.transform(X_test_imp)
X_all_scaled   = scaler.transform(X_all_imp)

# --- Optional PCA (fit on TRAIN only) ---
pca = None
if PCA_DIM is not None and isinstance(PCA_DIM, int) and 0 < PCA_DIM < X_train_scaled.shape[1]:
    pca = PCA(n_components=PCA_DIM, random_state=RANDOM_STATE)
    pca.fit(X_train_scaled)
    X_train_final = pca.transform(X_train_scaled)
    X_val_final   = pca.transform(X_val_scaled)
    X_test_final  = pca.transform(X_test_scaled)
    X_all_final   = pca.transform(X_all_scaled)
else:
    X_train_final = X_train_scaled
    X_val_final   = X_val_scaled
    X_test_final  = X_test_scaled
    X_all_final   = X_all_scaled

# --- KMeans fit on TRAIN only (K_FIXED) ---
if VERBOSE:
    print(f"Fitting KMeans with K={K_FIXED} on TRAIN (n={X_train_final.shape[0]} samples)...")
kmeans = KMeans(n_clusters=K_FIXED, random_state=RANDOM_STATE, n_init=10)
kmeans.fit(X_train_final)

# --- Predict cluster labels for train/val/test (nearest-centroid) ---
all_preds = kmeans.predict(X_all_final)

n_tr = len(df_train)
n_val = len(df_val)
n_te = len(df_test)
assert len(all_preds) == (n_tr + n_val + n_te)

df_train2 = df_train.reset_index(drop=True).copy()
df_val2   = df_val.reset_index(drop=True).copy()
df_test2  = df_test.reset_index(drop=True).copy()

df_train2["cluster"] = all_preds[:n_tr]
df_val2["cluster"]   = all_preds[n_tr:n_tr+n_val]
df_test2["cluster"]  = all_preds[n_tr+n_val:]

# --- Combine and save cluster assignment (useful for next blocks) ---
df_clusters_assigned = pd.concat([df_train2, df_val2, df_test2], ignore_index=True)
out_csv = os.path.join(OUT_DIR, "df_clusters_assigned_K10.csv")
df_clusters_assigned.to_csv(out_csv, index=False, encoding="utf-8-sig")
if VERBOSE:
    print("Saved cluster assignments to:", out_csv)

# --- Also produce a compact df_clusters (ticker,date,cluster,month) for Block7 tensor building ---
df_clusters = df_clusters_assigned[["ticker","date","cluster"]].copy()
df_clusters["month"] = df_clusters["date"].dt.to_period("M")
df_clusters_out = os.path.join(OUT_DIR, "df_clusters_for_tensors_K10.csv")
df_clusters.to_csv(df_clusters_out, index=False, encoding="utf-8-sig")
if VERBOSE:
    print("Saved df_clusters_for_tensors to:", df_clusters_out)

# --- Save pipeline (imputer, scaler, pca, kmeans, feature_cols) for reuse ---
pipe = {
    "imputer": imp,
    "scaler": scaler,
    "pca": pca,
    "kmeans": kmeans,
    "feature_cols": feature_cols,
    "train_range": (df_train["date"].min(), df_train["date"].max())
}
joblib.dump(pipe, os.path.join(OUT_DIR, f"cluster_pipeline_K{K_FIXED}.joblib"))
if VERBOSE:
    print("Saved pipeline to:", os.path.join(OUT_DIR, f"cluster_pipeline_K{K_FIXED}.joblib"))

# --- Diagnostics ---
if VERBOSE:
    print("Cluster counts (overall):")
    print(df_clusters_assigned["cluster"].value_counts().sort_index())
    print("Train cluster counts:")
    print(df_train2["cluster"].value_counts().sort_index())
    print("Example rows (clusters):")
    print(df_clusters_assigned[["ticker","date","cluster"]].head())

# cleanup
gc.collect()

Features used for clustering: ['Debt/Equity', 'Net Profit Margin (%)', 'P/E', 'P/B', 'ema_5_z', 'ema_20_z', 'ema_50_z', 'macd_z', 'macd_signal_z', 'macd_diff_z', 'rsi_z', 'bollinger_hband_z', 'bollinger_lband_z', 'atr_z', 'obv_z', 'vwap_z']
Train/Val/Test shapes: (95000, 18) (96915, 18) (72188, 18)
Fitting KMeans with K=10 on TRAIN (n=95000 samples)...
Saved cluster assignments to: ./clusters/df_clusters_assigned_K10.csv
Saved df_clusters_for_tensors to: ./clusters/df_clusters_for_tensors_K10.csv
Saved pipeline to: ./clusters/cluster_pipeline_K10.joblib
Cluster counts (overall):
cluster
0    36093
1    24082
2    15667
3    23544
4    46406
5    18496
6    25158
7    24518
8    20265
9    29874
Name: count, dtype: int64
Train cluster counts:
cluster
0     9593
1    17201
2     8306
3     7842
4    10192
5    10874
6     8078
7     9160
8     3216
9    10538
Name: count, dtype: int64
Example rows (clusters):
  ticker       date  cluster
0    AAA 2023-01-04        1
1    AAA 2023-01-05  

3132

**Block 7: Xây tensors (clusters mapping) và masks (active stocks)**

In [ ]:
# Block 7 — Tensors & Masks (fixed & compatible)
import os, gc, json
import numpy as np
import pandas as pd
from pathlib import Path

# ---------- Config ----------
LOOKBACK = 20
# sửa path (không có khoảng trắng), mặc định dùng cwd nếu C: không hợp lệ
BASE_DIR = Path("C:/tensors_out")
if not BASE_DIR.exists():
    BASE_DIR = Path("./tensors_out")
BASE_DIR.mkdir(parents=True, exist_ok=True)
SPLITS = ["train", "val", "test"]   # will create subfolders per split

print("Tensor base dir:", BASE_DIR.resolve())

# ---------- Load inputs (try workspace vars, else files) ----------
# Expecting: df_clusters_assigned (has columns ['ticker','date','cluster','split']) and df_features (has 'date','ticker',feature columns)
if "df_clusters_assigned" in globals():
    df_clusters_all = df_clusters_assigned.copy()
else:
    # try to load from cluster outputs folder
    candidate = Path("./clusters/df_clusters_for_tensors_K10.csv")
    candidate2 = Path("./clusters/df_clusters_assigned_K10.csv")
    if candidate.exists():
        df_clusters_all = pd.read_csv(candidate, parse_dates=["date"])
        print("Loaded clusters from", candidate)
    elif candidate2.exists():
        df_clusters_all = pd.read_csv(candidate2, parse_dates=["date"])
        print("Loaded clusters from", candidate2)
    else:
        raise ValueError("Cannot find df_clusters_assigned in workspace or './clusters/df_clusters_for_tensors_K10.csv'. Please run Block 6 first.")

# df_features: prefer workspace var, else load files saved by Block5
if "df_features" in globals():
    df_features = df_features.copy()
else:
    # attempt load train/val/test feature files and concat
    feat_dir = Path("./features")
    train_p = feat_dir / "df_features_train.csv"
    val_p   = feat_dir / "df_features_val.csv"
    test_p  = feat_dir / "df_features_test.csv"
    parts = []
    for p in [train_p, val_p, test_p]:
        if p.exists():
            parts.append(pd.read_csv(p, parse_dates=["date"]))
    if parts:
        df_features = pd.concat(parts, ignore_index=True).sort_values(["ticker","date"]).reset_index(drop=True)
        print("Loaded df_features from files in ./features/")
    else:
        raise ValueError("df_features not found in workspace and ./features/*.csv missing. Run Block5 first.")

# ---------- Normalize column names ----------
# ensure 'date' and 'ticker' exist in both
def ensure_date_col(df, col_candidates=("date","timestamp")):
    for c in col_candidates:
        if c in df.columns:
            if c != "date":
                df = df.rename(columns={c:"date"})
            df["date"] = pd.to_datetime(df["date"])
            return df
    raise ValueError("Dataframe must contain a date column named one of: " + ", ".join(col_candidates))

df_clusters_all = ensure_date_col(df_clusters_all)
df_features     = ensure_date_col(df_features)

# If df_clusters_all lacks 'split' column, try to infer from date ranges in feature split files
if "split" not in df_clusters_all.columns:
    # try to infer using known split date ranges (if available as files)
    # fallback: mark all as 'train' (user should provide df_clusters_assigned if they want exact splits)
    print("[WARN] df_clusters missing 'split' column. Attempting to infer by dates if possible.")
    # basic inference using features file min/max: require train/val/test files exist
    train_p = Path("./features/df_features_train.csv")
    val_p   = Path("./features/df_features_val.csv")
    test_p  = Path("./features/df_features_test.csv")
    if train_p.exists() and val_p.exists() and test_p.exists():
        tr = pd.read_csv(train_p, parse_dates=["date"])
        va = pd.read_csv(val_p, parse_dates=["date"])
        te = pd.read_csv(test_p, parse_dates=["date"])
        def assign_split(row):
            d = row["date"]
            if (d >= tr["date"].min()) and (d <= tr["date"].max()):
                return "train"
            if (d >= va["date"].min()) and (d <= va["date"].max()):
                return "val"
            if (d >= te["date"].min()) and (d <= te["date"].max()):
                return "test"
            return "train"
        df_clusters_all["split"] = df_clusters_all.apply(assign_split, axis=1)
        print("Inferred split column from ./features/*.csv ranges.")
    else:
        df_clusters_all["split"] = "train"
        print("Defaulted split='train' for all rows (no split files found).")

# ---------- Determine feature columns (exclude meta) ----------
meta_cols = {"ticker","date","cluster","split","month"}
# ensure df_features has 'ticker'
if "ticker" not in df_features.columns:
    raise ValueError("df_features must contain 'ticker' column.")
feature_cols = [c for c in df_features.columns if c not in meta_cols]
if len(feature_cols) == 0:
    raise ValueError("No feature columns found in df_features. Check Block 5 output.")

print(f"Found {len(feature_cols)} feature columns. Example: {feature_cols[:8]}")

# ---------- Helper functions ----------
def safe_name(s: str) -> str:
    # keep alnum and dash/underscore only
    return "".join(ch if (ch.isalnum() or ch in ("-","")) else "" for ch in str(s))

# ---------- Prepare output containers ----------
tensor_index = []

# For speed, convert df_features to multiindex keyed by (ticker,date) for fast join
df_features_indexed = df_features.set_index(["ticker","date"])

# ---------- Iterate splits/clusters ----------
for split in SPLITS:
    split_dir = BASE_DIR / split
    split_dir.mkdir(parents=True, exist_ok=True)

    df_c_split = df_clusters_all[df_clusters_all["split"] == split].copy()
    if df_c_split.empty:
        print(f"[WARN] No clusters for split={split}, skipping.")
        continue

    # group by cluster
    for c_id, g in df_c_split.groupby("cluster"):
        if int(c_id) == -1:
            continue
        tickers = sorted(g["ticker"].unique())
        if len(tickers) == 0:
            continue

        # Build list of unique (ticker,date) pairs for this cluster in this split
        key = g[["ticker","date"]].drop_duplicates()
        # join with df_features using index to keep feature columns aligned
        joined_rows = []
        for _, row in key.iterrows():
            tk = row["ticker"]; dt = pd.to_datetime(row["date"])
            try:
                feat_row = df_features_indexed.loc[(tk, dt)]
                # feat_row can be Series or DataFrame if duplicates; coerce to Series
                if isinstance(feat_row, pd.DataFrame):
                    feat_row = feat_row.iloc[0]
                rec = {"ticker": tk, "date": dt}
                for f in feature_cols:
                    rec[f] = feat_row.get(f, np.nan)
                joined_rows.append(rec)
            except KeyError:
                # missing feature row -> create NaNs
                rec = {"ticker": tk, "date": dt}
                for f in feature_cols:
                    rec[f] = np.nan
                joined_rows.append(rec)

        if len(joined_rows) == 0:
            print(f"[INFO] cluster {c_id} ({split}) no joined rows -> skip")
            continue

        df_join = pd.DataFrame(joined_rows)
        # pivot into (T, N, F)
        dates = sorted(df_join["date"].unique())
        if len(dates) <= LOOKBACK:
            print(f"[INFO] cluster {c_id} ({split}) has {len(dates)} dates <= LOOKBACK({LOOKBACK}), skipping.")
            continue

        N = len(tickers); T = len(dates); F = len(feature_cols)
        X = np.full((T, N, F), np.nan, dtype=np.float32)

        # fill X: for each feature fill pivot table values
        # create a mapping ticker -> col index
        tk2idx = {tk:i for i,tk in enumerate(tickers)}
        date2idx = {d:i for i,d in enumerate(dates)}

        for _, r in df_join.iterrows():
            ti = tk2idx[r["ticker"]]
            di = date2idx[pd.to_datetime(r["date"])]
            for fi, f in enumerate(feature_cols):
                val = r.get(f, np.nan)
                X[di, ti, fi] = np.float32(val) if not pd.isna(val) else np.nan

        # mask
        M = (~np.isnan(X)).astype(np.uint8)

        # forward/backfill along time per (ticker,feature)
        for n in range(N):
            for fi in range(F):
                col = X[:, n, fi]
                s = pd.Series(col, index=dates)
                s = s.ffill().bfill()
                X[:, n, fi] = s.values
                M[:, n, fi] = (~np.isnan(X[:, n, fi])).astype(np.uint8)

        # sliding windows of LOOKBACK -> B = T - LOOKBACK
        cluster_tensors = []
        cluster_masks   = []
        cluster_dates   = []
        for i in range(LOOKBACK, T):
            wX = X[i-LOOKBACK:i, :, :].copy()
            wM = M[i-LOOKBACK:i, :, :].copy()
            # skip windows entirely NaN
            if np.all(wM == 0):
                continue
            cluster_tensors.append(wX)
            cluster_masks.append(wM)
            cluster_dates.append(dates[i])  # window end date

        if len(cluster_tensors) == 0:
            print(f"[INFO] cluster {c_id} ({split}) produced 0 windows, skipping.")
            continue

        X_arr = np.stack(cluster_tensors, axis=0).astype(np.float32)  # (B, LOOKBACK, N, F)
        M_arr = np.stack(cluster_masks, axis=0).astype(np.uint8)

        # filenames
        safe_cluster = safe_name(c_id)
        tfile = f"{split}_cluster_{safe_cluster}_tensor.npy"
        mfile = f"{split}_cluster_{safe_cluster}_mask.npy"
        tpath = split_dir / tfile
        mpath = split_dir / mfile

        np.save(str(tpath), X_arr)
        np.save(str(mpath), M_arr)

        tensor_index.append({
            "split": split,
            "cluster": int(c_id),
            "tickers": tickers,
            "n_windows": int(X_arr.shape[0]),
            "window_size": int(LOOKBACK),
            "tensor_file": str(tpath.resolve()),
            "mask_file": str(mpath.resolve()),
            "dates": [d.isoformat() for d in cluster_dates]
        })

        print(f"[Saved] split={split} cluster={c_id} windows={X_arr.shape[0]} shape={X_arr.shape}")

        # free mem
        del X_arr, M_arr, X, M, cluster_tensors, cluster_masks
        gc.collect()

# Save tensor_index
index_path = BASE_DIR / "tensor_index.json"
with open(index_path, "w", encoding="utf-8") as fh:
    json.dump(tensor_index, fh, indent=2, ensure_ascii=False)

print("✅ Done Block 7: tensors saved. Index:", index_path.resolve())

Tensor base dir: C:\tensors_out
[WARN] df_clusters missing 'split' column. Attempting to infer by dates if possible.
Inferred split column from ./features/*.csv ranges.
Found 16 feature columns. Example: ['Debt/Equity', 'Net Profit Margin (%)', 'P/E', 'P/B', 'ema_5_z', 'ema_20_z', 'ema_50_z', 'macd_z']
[Saved] split=train cluster=0 windows=193 shape=(193, 20, 350, 16)
[Saved] split=train cluster=1 windows=152 shape=(152, 20, 367, 16)
[Saved] split=train cluster=2 windows=169 shape=(169, 20, 326, 16)
[Saved] split=train cluster=3 windows=209 shape=(209, 20, 372, 16)
[Saved] split=train cluster=4 windows=66 shape=(66, 20, 356, 16)
[Saved] split=train cluster=5 windows=196 shape=(196, 20, 362, 16)
[Saved] split=train cluster=6 windows=212 shape=(212, 20, 317, 16)
[Saved] split=train cluster=7 windows=205 shape=(205, 20, 373, 16)
[Saved] split=train cluster=8 windows=228 shape=(228, 20, 23, 16)
[Saved] split=train cluster=9 windows=206 shape=(206, 20, 366, 16)
[Saved] split=val cluster=0 w

**Block 7.5: Chuẩn bị dữ liệu backtest(loại bỏ các cột dữ liệu không cần thiết nữa)**

In [ ]:
# Block 7.5 — Chuẩn bị dữ liệu backtest cho reward thật (từ CSV Block 3) — FIXED
import os, gc
import pandas as pd
import numpy as np

# --- Config ---
merged_file = "vnindex_price_fa_merged.csv"
OUT_DIR = "./backtest_ddpg/"
OUT_FILE = os.path.join(OUT_DIR, "df_backtest.csv")

# --- Load kiểm tra ---
if not os.path.exists(merged_file):
    raise FileNotFoundError(f"Không tìm thấy '{merged_file}'. Hãy chạy lại Block 3 trước.")

# đọc file, chấp nhận cả 'timestamp' hoặc 'date'
df_price = pd.read_csv(merged_file, low_memory=False)
# chuẩn hoá column ngày
if "timestamp" in df_price.columns and "date" not in df_price.columns:
    df_price = df_price.rename(columns={"timestamp": "date"})
if "date" not in df_price.columns:
    raise ValueError("File merged phải có cột 'timestamp' hoặc 'date'")

df_price["date"] = pd.to_datetime(df_price["date"], errors="coerce")
# drop rows có date NaT
n_bad_date = df_price["date"].isna().sum()
if n_bad_date > 0:
    print(f"[WARN] Bỏ {n_bad_date} dòng có date không hợp lệ.")
    df_price = df_price[df_price["date"].notna()].copy()

# --- Kiểm tra presence của OHLC ---
required = ["ticker","date","open","high","low","close"]
miss = [c for c in required if c not in df_price.columns]
if miss:
    raise ValueError(f"Price Data phải chứa các cột: {', '.join(required)}. Thiếu: {', '.join(miss)}")

# --- Ép kiểu số cho giá (loại bỏ giá không hợp lệ) ---
for c in ["open","high","low","close"]:
    # replace commas, whitespace nếu có rồi to_numeric
    df_price[c] = pd.to_numeric(df_price[c], errors="coerce")

# Remove rows where all four prices are NaN (bad rows)
keep_mask = ~(df_price[["open","high","low","close"]].isna().all(axis=1))
n_removed = (~keep_mask).sum()
if n_removed > 0:
    print(f"[WARN] Loại {n_removed} dòng không có giá hợp lệ trong open/high/low/close.")
    df_price = df_price[keep_mask].copy()

# --- Giữ các cột cần thiết và sort ---
cols_needed = ["ticker","date","open","high","low","close"]
df_backtest = df_price[cols_needed].copy()
df_backtest = df_backtest.sort_values(["date","ticker"]).reset_index(drop=True)

# --- Optional: forward/backfill small gaps per ticker (commented out, bật nếu muốn) ---
# df_backtest[['open','high','low','close']] = df_backtest.groupby('ticker')[['open','high','low','close']].apply(lambda g: g.ffill().bfill())

# --- Summary & save ---
os.makedirs(OUT_DIR, exist_ok=True)
df_backtest.to_csv(OUT_FILE, index=False, encoding="utf-8-sig")

print("✅ Done Block 7.5: df_backtest sẵn sàng cho reward/SL/TP.")
print("  Saved to:", OUT_FILE)
print("  Shape:", df_backtest.shape)
print("  Unique tickers:", int(df_backtest['ticker'].nunique()))
print("  Timeline:", df_backtest['date'].min().date(), "→", df_backtest['date'].max().date())


✅ Done Block 7.5: df_backtest sẵn sàng cho reward/SL/TP.
  Saved to: ./backtest_ddpg/df_backtest.csv
  Shape: (546084, 6)
  Unique tickers: 391
  Timeline: 2020-01-02 → 2025-10-02


**Block 8: Huấn luyện A3C theo từng cụm**

In [110]:
# Block 8 — A3C per-cluster (full, stable, val-checkpoint, inference)
import os, gc, json, csv, math, time
from pathlib import Path
from typing import List, Dict
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# =========== CONFIG / PATH ===========
BASE_TENSOR_DIR = Path("C:/tensors_out")         # nơi có train/val/test subfolders + tensor_index.json
TENSOR_INDEX = BASE_TENSOR_DIR / "tensor_index.json"
DF_BACKTEST = Path("./backtest_ddpg/df_backtest.csv")   # nếu có -> dùng để tính reward next-day
SIG_DIR = Path("./signals/")
MODEL_DIR = Path("./models_a3c/")
SIG_TRAIN_FILE = SIG_DIR / "a3c_signals_train.csv"
SIG_INFER_FILE = SIG_DIR / "a3c_signals_infer.csv"

SIG_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# =========== HYPER (tune) ===========
EPOCHS = 6               # epochs per cluster (increase later)
BATCH_SIZE = 256
LR = 1e-4
HIDDEN = 128
ENTROPY_BETA = 0.01
CRITIC_COEF = 0.5
GRAD_CLIP = 1.0
REWARD_SCALE = 1.0
SEED = 42
LOOKBACK_EXPECT = None   # None = inferred per-cluster
torch.manual_seed(SEED); np.random.seed(SEED)

# =========== DEVICE ===========
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# =========== SAFETY helpers ===========
EPS = 1e-8

def safe_tensor_np(x, dtype=np.float32):
    a = np.array(x, dtype=dtype)
    # replace inf/nan
    a = np.nan_to_num(a, nan=0.0, posinf=0.0, neginf=0.0)
    return a

def safe_tensor(x, dtype=torch.float32):
    t = torch.tensor(np.array(x), dtype=dtype, device=DEVICE)
    t = torch.where(torch.isfinite(t), t, torch.zeros_like(t))
    return t

def is_bad(t: torch.Tensor):
    return torch.isnan(t).any().item() or torch.isinf(t).any().item()

# =========== MODEL ===========
class A3CNet(nn.Module):
    def __init__(self, n_features, hidden=HIDDEN):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=hidden, batch_first=True)
        self.actor = nn.Sequential(
            nn.Linear(hidden, max(32, hidden//2)),
            nn.ReLU(),
            nn.Linear(max(32, hidden//2), 3)   # logits: short(0), flat(1), long(2)
        )
        self.critic = nn.Sequential(
            nn.Linear(hidden, max(32, hidden//2)),
            nn.ReLU(),
            nn.Linear(max(32, hidden//2), 1)
        )
    def forward(self, x):
        # x: (B, T, F)
        out, _ = self.lstm(x)
        h = out[:, -1, :]                          # (B, hidden)
        logits = self.actor(h)                     # (B, 3)
        value = self.critic(h).squeeze(-1)         # (B,)
        return logits, value

# =========== IO helpers (split-aware) ===========
def resolve_path_rel_to_split(path_str: str, split: str):
    """
    If given path_str exists -> return Path(path_str).
    Else, attempt BASE_TENSOR_DIR / split / basename(path_str).
    """
    p = Path(path_str)
    if p.exists():
        return p
    candidate = BASE_TENSOR_DIR / split / p.name
    if candidate.exists():
        return candidate
    # fallback: try under base dir directly
    candidate2 = BASE_TENSOR_DIR / p.name
    if candidate2.exists():
        return candidate2
    # final fallback: return expected candidate (may raise later)
    return candidate

# =========== LOAD tensor_index ===========
if not TENSOR_INDEX.exists():
    raise FileNotFoundError(f"tensor_index.json not found at {TENSOR_INDEX}. Run Block 7 first.")
with open(TENSOR_INDEX, "r", encoding="utf-8") as fh:
    tensor_index_all = json.load(fh)

# =========== LOAD price (backtest) for reward if available ===========
USE_PRICE_REWARD = DF_BACKTEST.exists()
if USE_PRICE_REWARD:
    df_back = pd.read_csv(DF_BACKTEST, parse_dates=["date"])
    df_back = df_back.sort_values(["ticker","date"]).reset_index(drop=True)
    px_close = df_back.pivot(index="date", columns="ticker", values="close").sort_index()
    print("df_backtest loaded: dates", px_close.index.min(), "->", px_close.index.max())
else:
    px_close = None
    print("No df_backtest.csv found -> training with synthetic/zero reward unless user supplies df_backtest.csv")

# ---------- helper: compute next-day log return for arrays of dates & tickers ----------
def compute_nextday_logret(dates_iso: List[str], tickers: List[str]):
    """
    dates_iso: list of window-end dates (ISO strings). returns array shape (B, N)
    For each date d, compute log(close[next_trading_day, ticker] / close[d, ticker])
    If px_close missing -> return zeros.
    """
    if px_close is None:
        return np.zeros((len(dates_iso), len(tickers)), dtype=np.float32)

    d_idx = pd.to_datetime(dates_iso)
    # find nearest index positions (<= d), then pos+1 -> next trading day if exists
    all_index = px_close.index
    res = np.zeros((len(d_idx), len(tickers)), dtype=np.float32)
    for i, d in enumerate(d_idx):
        # get location of nearest index <= d
        pos = all_index.get_indexer([d], method='pad')[0]
        pos_next = pos + 1 if (pos + 1) < len(all_index) else pos
        date_curr = all_index[pos]
        date_next = all_index[pos_next]
        # slice
        v_curr = px_close.loc[date_curr, tickers].values.astype(np.float32)
        v_next = px_close.loc[date_next, tickers].values.astype(np.float32)
        # safe division
        with np.errstate(divide='ignore', invalid='ignore'):
            r = np.log(np.divide(v_next, np.maximum(v_curr, 1e-9)))
        r = np.nan_to_num(r, nan=0.0, posinf=0.0, neginf=0.0)
        res[i, :] = r
    return res

# =========== safe loader for tensor meta ===========
def load_tensor_meta(meta: Dict):
    """
    meta expected keys: split, cluster, tensor_file, mask_file, tickers, dates (list), dates_shifted (list optional)
    returns: X (B, LOOKBACK, N, F), M (B, LOOKBACK, N, F), tickers(list), dates(list), split(str)
    """
    split = meta.get("split", "train")
    tpath = resolve_path_rel_to_split(meta["tensor_file"], split)
    mpath = resolve_path_rel_to_split(meta["mask_file"], split)
    if not tpath.exists() or not mpath.exists():
        raise FileNotFoundError(f"Tensor/Mask not found for cluster {meta.get('cluster')} split={split}: {tpath}, {mpath}")
    X = np.load(tpath, mmap_mode="r")
    M = np.load(mpath, mmap_mode="r")
    tickers = list(meta.get("tickers", []))
    dates = [pd.to_datetime(d) for d in meta.get("dates", [])]
    dates_shifted = [pd.to_datetime(d) for d in meta.get("dates_shifted", [])] if meta.get("dates_shifted") else None
    return X, M, tickers, dates, dates_shifted, split

# =========== Training per-cluster (with val checkpointing) ===========
def train_cluster(meta, epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR, save_dir=MODEL_DIR):
    cluster_id = int(meta.get("cluster"))
    split = meta.get("split", "train")
    if split != "train":
        print(f"Skipping non-train meta cluster {cluster_id} (split={split})")
        return None
    print(f"\n--- TRAIN cluster {cluster_id} (split=train) ---")
    # load train meta
    try:
        X_tr, M_tr, tickers_tr, dates_tr, dates_shifted_tr, _ = load_tensor_meta(meta)
    except Exception as e:
        print("  [ERROR] load tensor:", e)
        return None
    B, LOOKBACK, N, F = X_tr.shape
    print(f"  windows={B}, LOOKBACK={LOOKBACK}, N={N}, F={F}")

    # find val meta for same cluster if exists
    val_meta = None
    for m in tensor_index_all:
        if int(m.get("cluster"))==cluster_id and m.get("split","")=="val":
            val_meta = m; break
    if val_meta:
        X_val, M_val, tickers_val, dates_val, dates_shifted_val, _ = load_tensor_meta(val_meta)
        print(f"  Found val windows={X_val.shape[0]}")
        # precompute val rewards
        r_val = compute_nextday_logret([d.isoformat() for d in dates_val], tickers_val) * REWARD_SCALE
    else:
        X_val = M_val = tickers_val = dates_val = dates_shifted_val = None
        r_val = None

    # precompute train rewards (B, N)
    r_tr = compute_nextday_logret([d.isoformat() for d in dates_tr], tickers_tr) * REWARD_SCALE

    # safe conversions
    X_tr = np.nan_to_num(X_tr, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    M_tr = np.nan_to_num(M_tr, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)

    # model & optimizer
    model = A3CNet(n_features=F, hidden=HIDDEN).to(DEVICE)
    opt = optim.Adam(model.parameters(), lr=lr)

    best_val_score = -1e9
    best_path = save_dir / f"a3c_cluster_{cluster_id}_best.pt"
    last_path = save_dir / f"a3c_cluster_{cluster_id}_last.pt"

    total_samples = B * N
    indices = np.arange(total_samples)

    def batch_iter(shuffle=True):
        if shuffle:
            np.random.shuffle(indices)
        for start in range(0, total_samples, batch_size):
            end = min(total_samples, start + batch_size)
            batch_idx = indices[start:end]
            xb_list, mb_list, rb_list = [], [], []
            for s in batch_idx:
                b, n = divmod(int(s), N)
                seq = X_tr[b, :, n, :]          # (LOOKBACK, F)
                mask = M_tr[b, :, n, :]         # (LOOKBACK, F)
                xb_list.append(seq)
                mb_list.append(mask)
                rb_list.append(r_tr[b, n])
            yield np.stack(xb_list, axis=0), np.stack(mb_list, axis=0), np.array(rb_list, dtype=np.float32), batch_idx

    # training loop
    for ep in range(1, epochs+1):
        model.train()
        ep_loss = 0.0; ep_actor = 0.0; ep_critic = 0.0; ep_ent = 0.0; ep_batches = 0
        for xb_np, mb_np, rb_np, batch_idx in batch_iter(shuffle=True):
            # to tensors
            xb = safe_tensor(xb_np); mb = safe_tensor(mb_np); rb = safe_tensor(rb_np)
            # apply mask to inputs
            xb = xb * mb
            # forward
            logits, vals = model(xb)                 # logits (B,3), vals (B,)
            if is_bad(logits) or is_bad(vals) or is_bad(rb):
                print("  [WARN] bad numerics in forward -> skip batch")
                continue
            dist = torch.distributions.Categorical(logits=logits)
            acts = dist.sample()                     # (B,)
            logp = torch.log_softmax(logits, dim=-1)
            logp_act = logp.gather(1, acts.unsqueeze(1)).squeeze(1)

            # map action -> reward: long(2) => +r, short(0) => -r, flat => 0
            reward_t = torch.where(acts==2, rb, torch.where(acts==0, -rb, torch.zeros_like(rb)))

            # advantage
            adv = reward_t - vals
            adv_mean = adv.mean()
            adv_std = adv.std(unbiased=False) + EPS
            adv_norm = (adv - adv_mean) / adv_std

            actor_loss = -(logp_act * adv_norm.detach()).mean()
            critic_loss = CRITIC_COEF * (adv.pow(2).mean())
            entropy = - (torch.softmax(logits, dim=-1) * logp).sum(-1).mean()
            loss = actor_loss + critic_loss - ENTROPY_BETA * entropy

            if not math.isfinite(float(loss.item())):
                print("  [WARN] loss not finite -> skip batch")
                continue

            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            opt.step()

            ep_loss += float(loss.item()); ep_actor += float(actor_loss.item())
            ep_critic += float(critic_loss.item()); ep_ent += float(entropy.item()); ep_batches += 1

            # free small
            del xb, mb, rb, logits, vals, dist, acts, logp, logp_act, reward_t, adv, adv_norm
            gc.collect()

        if ep_batches == 0:
            print(f"  [WARN] no batches processed in epoch {ep}")
            continue

        print(f"[Cluster {cluster_id}] Ep {ep}/{epochs} | loss={ep_loss/ep_batches:.6f} actor={ep_actor/ep_batches:.6f} critic={ep_critic/ep_batches:.6f} ent={ep_ent/ep_batches:.6f}")

        # validation evaluation (deterministic argmax if val exists)
        if X_val is not None:
            model.eval()
            # build val inference in batches
            val_total = X_val.shape[0] * X_val.shape[2]
            val_batchsize = batch_size
            val_rewards = []
            for start in range(0, val_total, val_batchsize):
                xb_list = []; rb_list = []
                for s in range(start, min(val_total, start+val_batchsize)):
                    b, n = divmod(int(s), X_val.shape[2])
                    seq = X_val[b, :, n, :]
                    seq = np.nan_to_num(seq, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
                    xb_list.append(seq)
                    rb_list.append(r_val[b, n])
                if not xb_list:
                    continue
                xb_t = safe_tensor(np.stack(xb_list, axis=0))
                logits_v, vals_v = model(xb_t)
                acts_v = torch.argmax(logits_v, dim=-1)
                rb_t = safe_tensor(np.array(rb_list, dtype=np.float32))
                rdet = torch.where(acts_v==2, rb_t, torch.where(acts_v==0, -rb_t, torch.zeros_like(rb_t)))
                val_rewards.append(rdet.cpu().numpy())
                del xb_t, logits_v, vals_v, acts_v, rb_t, rdet
            if val_rewards:
                val_arr = np.concatenate(val_rewards, axis=0)
                val_score = float(val_arr.mean())
            else:
                val_score = -1e9
            print(f"  [VAL] deterministic mean net reward = {val_score:.6f}")
            if val_score > best_val_score:
                best_val_score = val_score
                torch.save(model.state_dict(), best_path)
                print(f"  ✅ New best checkpoint saved for cluster {cluster_id} (val={val_score:.6f})")

        # always save last
        torch.save(model.state_dict(), last_path)

    # end epochs
    print(f"--- Finished training cluster {cluster_id}. Best val score = {best_val_score:.6f} ---")
    ckpt = best_path if best_path.exists() else last_path
    return str(ckpt)

# =========== Inference (deterministic argmax) ===========
def infer_on_split(output_csv: str, split_name: str = "test"):
    # reset file
    if os.path.exists(output_csv):
        os.remove(output_csv)
    with open(output_csv, "w", newline="") as fh:
        csv.writer(fh).writerow(["date","ticker","signal"])
    for meta in tensor_index_all:
        if meta.get("split") != split_name:
            continue
        cluster_id = int(meta.get("cluster"))
        print(f"[INF] cluster={cluster_id} split={split_name}")
        try:
            X, M, tickers, dates, dates_shifted, split = load_tensor_meta(meta)
        except Exception as e:
            print("  [WARN] load tensor failed:", e); continue
        B, LOOKBACK, N, F = X.shape
        # choose checkpoint: best else last
        best_path = MODEL_DIR / f"a3c_cluster_{cluster_id}_best.pt"
        last_path = MODEL_DIR / f"a3c_cluster_{cluster_id}_last.pt"
        model_path = best_path if best_path.exists() else last_path if last_path.exists() else None
        if model_path is None:
            print(f"  [WARN] no checkpoint for cluster {cluster_id}, skipping")
            continue
        model = A3CNet(n_features=F, hidden=HIDDEN).to(DEVICE)
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        model.eval()
        total = B * N
        with open(output_csv, "a", newline="") as fh:
            w = csv.writer(fh)
            for start in range(0, total, BATCH_SIZE):
                xb_list = []; idx_list = []
                for s in range(start, min(total, start+BATCH_SIZE)):
                    b, n = divmod(int(s), N)
                    seq = X[b, :, n, :]
                    seq = np.nan_to_num(seq, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
                    xb_list.append(seq); idx_list.append((b,n))
                if not xb_list:
                    continue
                xb_t = safe_tensor(np.stack(xb_list, axis=0))
                logits, vals = model(xb_t)
                acts = torch.argmax(logits, dim=-1).cpu().numpy()    # 0,1,2
                acts_mapped = np.where(acts==2, 1, np.where(acts==0, -1, 0))
                for k, (b,n) in enumerate(idx_list):
                    out_date = dates_shifted[b].isoformat() if dates_shifted is not None and len(dates_shifted)>b else pd.to_datetime(dates[b]).date().isoformat()
                    w.writerow([out_date, tickers[n], int(acts_mapped[k])])
                del xb_t, logits, vals, acts
                gc.collect()
        print(f"  [INF] cluster {cluster_id} -> appended signals to {output_csv}")

# =========== MAIN flow ===========
# 1) Train clusters that have split=train entries (iterate over tensor_index_all)
trained = {}
for meta in tensor_index_all:
    if meta.get("split") != "train":
        continue
    try:
        ckpt = train_cluster(meta, epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR)
        if ckpt:
            trained[int(meta.get("cluster"))] = ckpt
    except Exception as e:
        print(f"[ERROR] training cluster {meta.get('cluster')}: {e}")

# 2) After training, run inference on split=test and save CSV
print("\n=== Running inference on split=test and writing to", SIG_INFER_FILE, "===")
infer_on_split(str(SIG_INFER_FILE), split_name="test")
print("\n✅ Block 8 complete: training done and inference saved to", SIG_INFER_FILE)

Device: cpu
df_backtest loaded: dates 2020-01-02 00:00:00 -> 2025-10-02 00:00:00

--- TRAIN cluster 0 (split=train) ---
  windows=193, LOOKBACK=20, N=350, F=16
  Found val windows=230
[Cluster 0] Ep 1/6 | loss=-0.015854 actor=-0.005562 critic=0.000217 ent=1.050907
  [VAL] deterministic mean net reward = 0.000183
  ✅ New best checkpoint saved for cluster 0 (val=0.000183)
[Cluster 0] Ep 2/6 | loss=-0.014637 actor=-0.011404 critic=0.000258 ent=0.349111
  [VAL] deterministic mean net reward = 0.000183
[Cluster 0] Ep 3/6 | loss=-0.008478 actor=-0.007185 critic=0.000267 ent=0.155960
  [VAL] deterministic mean net reward = 0.000183
[Cluster 0] Ep 4/6 | loss=-0.005977 actor=-0.005140 critic=0.000269 ent=0.110622
  [VAL] deterministic mean net reward = 0.000183
[Cluster 0] Ep 5/6 | loss=-0.005695 actor=-0.004631 critic=0.000267 ent=0.133128
  [VAL] deterministic mean net reward = 0.000183
[Cluster 0] Ep 6/6 | loss=-0.005296 actor=-0.003683 critic=0.000261 ent=0.187431
  [VAL] deterministic mean

In [ ]:
# Block 8 — A3C per-cluster (full, stable, val-checkpoint, inference)
import os, gc, json, csv, math, time
from pathlib import Path
from typing import List, Dict
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# =========== CONFIG / PATH ===========
BASE_TENSOR_DIR = Path("C:/tensors_out")         # nơi có train/val/test subfolders + tensor_index.json
TENSOR_INDEX = BASE_TENSOR_DIR / "tensor_index.json"
DF_BACKTEST = Path("./backtest_ddpg/df_backtest.csv")   # nếu có -> dùng để tính reward next-day
SIG_DIR = Path("./signals/")
MODEL_DIR = Path("./models_a3c/")
SIG_TRAIN_FILE = SIG_DIR / "a3c_signals_train.csv"
SIG_INFER_FILE = SIG_DIR / "a3c_signals_infer.csv"

SIG_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# =========== HYPER (tune) ===========
EPOCHS = 3               # epochs per cluster (increase later)
BATCH_SIZE = 256
LR = 1e-4
HIDDEN = 128
ENTROPY_BETA = 0.01
CRITIC_COEF = 0.5
GRAD_CLIP = 1.0
REWARD_SCALE = 1.0
SEED = 42
LOOKBACK_EXPECT = None   # None = inferred per-cluster
torch.manual_seed(SEED); np.random.seed(SEED)

# =========== DEVICE ===========
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# =========== SAFETY helpers ===========
EPS = 1e-8

def safe_tensor_np(x, dtype=np.float32):
    a = np.array(x, dtype=dtype)
    # replace inf/nan
    a = np.nan_to_num(a, nan=0.0, posinf=0.0, neginf=0.0)
    return a

def safe_tensor(x, dtype=torch.float32):
    t = torch.tensor(np.array(x), dtype=dtype, device=DEVICE)
    t = torch.where(torch.isfinite(t), t, torch.zeros_like(t))
    return t

def is_bad(t: torch.Tensor):
    return torch.isnan(t).any().item() or torch.isinf(t).any().item()

# =========== MODEL ===========
class A3CNet(nn.Module):
    def __init__(self, n_features, hidden=HIDDEN):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=hidden, batch_first=True)
        self.actor = nn.Sequential(
            nn.Linear(hidden, max(32, hidden//2)),
            nn.ReLU(),
            nn.Linear(max(32, hidden//2), 3)   # logits: short(0), flat(1), long(2)
        )
        self.critic = nn.Sequential(
            nn.Linear(hidden, max(32, hidden//2)),
            nn.ReLU(),
            nn.Linear(max(32, hidden//2), 1)
        )
    def forward(self, x):
        # x: (B, T, F)
        out, _ = self.lstm(x)
        h = out[:, -1, :]                          # (B, hidden)
        logits = self.actor(h)                     # (B, 3)
        value = self.critic(h).squeeze(-1)         # (B,)
        return logits, value

# =========== IO helpers (split-aware) ===========
def resolve_path_rel_to_split(path_str: str, split: str):
    """
    If given path_str exists -> return Path(path_str).
    Else, attempt BASE_TENSOR_DIR / split / basename(path_str).
    """
    p = Path(path_str)
    if p.exists():
        return p
    candidate = BASE_TENSOR_DIR / split / p.name
    if candidate.exists():
        return candidate
    # fallback: try under base dir directly
    candidate2 = BASE_TENSOR_DIR / p.name
    if candidate2.exists():
        return candidate2
    # final fallback: return expected candidate (may raise later)
    return candidate

# =========== LOAD tensor_index ===========
if not TENSOR_INDEX.exists():
    raise FileNotFoundError(f"tensor_index.json not found at {TENSOR_INDEX}. Run Block 7 first.")
with open(TENSOR_INDEX, "r", encoding="utf-8") as fh:
    tensor_index_all = json.load(fh)

# =========== LOAD price (backtest) for reward if available ===========
USE_PRICE_REWARD = DF_BACKTEST.exists()
if USE_PRICE_REWARD:
    df_back = pd.read_csv(DF_BACKTEST, parse_dates=["date"])
    df_back = df_back.sort_values(["ticker","date"]).reset_index(drop=True)
    px_close = df_back.pivot(index="date", columns="ticker", values="close").sort_index()
    print("df_backtest loaded: dates", px_close.index.min(), "->", px_close.index.max())
else:
    px_close = None
    print("No df_backtest.csv found -> training with synthetic/zero reward unless user supplies df_backtest.csv")

# ---------- helper: compute next-day log return for arrays of dates & tickers ----------
def compute_nextday_logret(dates_iso: List[str], tickers: List[str]):
    """
    dates_iso: list of window-end dates (ISO strings). returns array shape (B, N)
    For each date d, compute log(close[next_trading_day, ticker] / close[d, ticker])
    If px_close missing -> return zeros.
    """
    if px_close is None:
        return np.zeros((len(dates_iso), len(tickers)), dtype=np.float32)

    d_idx = pd.to_datetime(dates_iso)
    # find nearest index positions (<= d), then pos+1 -> next trading day if exists
    all_index = px_close.index
    res = np.zeros((len(d_idx), len(tickers)), dtype=np.float32)
    for i, d in enumerate(d_idx):
        # get location of nearest index <= d
        pos = all_index.get_indexer([d], method='pad')[0]
        pos_next = pos + 1 if (pos + 1) < len(all_index) else pos
        date_curr = all_index[pos]
        date_next = all_index[pos_next]
        # slice
        v_curr = px_close.loc[date_curr, tickers].values.astype(np.float32)
        v_next = px_close.loc[date_next, tickers].values.astype(np.float32)
        # safe division
        with np.errstate(divide='ignore', invalid='ignore'):
            r = np.log(np.divide(v_next, np.maximum(v_curr, 1e-9)))
        r = np.nan_to_num(r, nan=0.0, posinf=0.0, neginf=0.0)
        res[i, :] = r
    return res

# =========== safe loader for tensor meta ===========
def load_tensor_meta(meta: Dict):
    """
    meta expected keys: split, cluster, tensor_file, mask_file, tickers, dates (list), dates_shifted (list optional)
    returns: X (B, LOOKBACK, N, F), M (B, LOOKBACK, N, F), tickers(list), dates(list), split(str)
    """
    split = meta.get("split", "train")
    tpath = resolve_path_rel_to_split(meta["tensor_file"], split)
    mpath = resolve_path_rel_to_split(meta["mask_file"], split)
    if not tpath.exists() or not mpath.exists():
        raise FileNotFoundError(f"Tensor/Mask not found for cluster {meta.get('cluster')} split={split}: {tpath}, {mpath}")
    X = np.load(tpath, mmap_mode="r")
    M = np.load(mpath, mmap_mode="r")
    tickers = list(meta.get("tickers", []))
    dates = [pd.to_datetime(d) for d in meta.get("dates", [])]
    dates_shifted = [pd.to_datetime(d) for d in meta.get("dates_shifted", [])] if meta.get("dates_shifted") else None
    return X, M, tickers, dates, dates_shifted, split

# =========== Training per-cluster (with val checkpointing) ===========
def train_cluster(meta, epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR, save_dir=MODEL_DIR):
    cluster_id = int(meta.get("cluster"))
    split = meta.get("split", "train")
    if split != "train":
        print(f"Skipping non-train meta cluster {cluster_id} (split={split})")
        return None
    print(f"\n--- TRAIN cluster {cluster_id} (split=train) ---")
    # load train meta
    try:
        X_tr, M_tr, tickers_tr, dates_tr, dates_shifted_tr, _ = load_tensor_meta(meta)
    except Exception as e:
        print("  [ERROR] load tensor:", e)
        return None
    B, LOOKBACK, N, F = X_tr.shape
    print(f"  windows={B}, LOOKBACK={LOOKBACK}, N={N}, F={F}")

    # find val meta for same cluster if exists
    val_meta = None
    for m in tensor_index_all:
        if int(m.get("cluster"))==cluster_id and m.get("split","")=="val":
            val_meta = m; break
    if val_meta:
        X_val, M_val, tickers_val, dates_val, dates_shifted_val, _ = load_tensor_meta(val_meta)
        print(f"  Found val windows={X_val.shape[0]}")
        # precompute val rewards
        r_val = compute_nextday_logret([d.isoformat() for d in dates_val], tickers_val) * REWARD_SCALE
    else:
        X_val = M_val = tickers_val = dates_val = dates_shifted_val = None
        r_val = None

    # precompute train rewards (B, N)
    r_tr = compute_nextday_logret([d.isoformat() for d in dates_tr], tickers_tr) * REWARD_SCALE

    # safe conversions
    X_tr = np.nan_to_num(X_tr, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    M_tr = np.nan_to_num(M_tr, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)

    # model & optimizer
    model = A3CNet(n_features=F, hidden=HIDDEN).to(DEVICE)
    opt = optim.Adam(model.parameters(), lr=lr)

    best_val_score = -1e9
    best_path = save_dir / f"a3c_cluster_{cluster_id}_best.pt"
    last_path = save_dir / f"a3c_cluster_{cluster_id}_last.pt"

    total_samples = B * N
    indices = np.arange(total_samples)

    def batch_iter(shuffle=True):
        if shuffle:
            np.random.shuffle(indices)
        for start in range(0, total_samples, batch_size):
            end = min(total_samples, start + batch_size)
            batch_idx = indices[start:end]
            xb_list, mb_list, rb_list = [], [], []
            for s in batch_idx:
                b, n = divmod(int(s), N)
                seq = X_tr[b, :, n, :]          # (LOOKBACK, F)
                mask = M_tr[b, :, n, :]         # (LOOKBACK, F)
                xb_list.append(seq)
                mb_list.append(mask)
                rb_list.append(r_tr[b, n])
            yield np.stack(xb_list, axis=0), np.stack(mb_list, axis=0), np.array(rb_list, dtype=np.float32), batch_idx

    # training loop
    for ep in range(1, epochs+1):
        model.train()
        ep_loss = 0.0; ep_actor = 0.0; ep_critic = 0.0; ep_ent = 0.0; ep_batches = 0
        for xb_np, mb_np, rb_np, batch_idx in batch_iter(shuffle=True):
            # to tensors
            xb = safe_tensor(xb_np); mb = safe_tensor(mb_np); rb = safe_tensor(rb_np)
            # apply mask to inputs
            xb = xb * mb
            # forward
            logits, vals = model(xb)                 # logits (B,3), vals (B,)
            if is_bad(logits) or is_bad(vals) or is_bad(rb):
                print("  [WARN] bad numerics in forward -> skip batch")
                continue
            dist = torch.distributions.Categorical(logits=logits)
            acts = dist.sample()                     # (B,)
            logp = torch.log_softmax(logits, dim=-1)
            logp_act = logp.gather(1, acts.unsqueeze(1)).squeeze(1)

            # map action -> reward: long(2) => +r, short(0) => -r, flat => 0
            reward_t = torch.where(acts==2, rb, torch.where(acts==0, -rb, torch.zeros_like(rb)))

            # advantage
            adv = reward_t - vals
            adv_mean = adv.mean()
            adv_std = adv.std(unbiased=False) + EPS
            adv_norm = (adv - adv_mean) / adv_std

            actor_loss = -(logp_act * adv_norm.detach()).mean()
            critic_loss = CRITIC_COEF * (adv.pow(2).mean())
            entropy = - (torch.softmax(logits, dim=-1) * logp).sum(-1).mean()
            loss = actor_loss + critic_loss - ENTROPY_BETA * entropy

            if not math.isfinite(float(loss.item())):
                print("  [WARN] loss not finite -> skip batch")
                continue

            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            opt.step()

            ep_loss += float(loss.item()); ep_actor += float(actor_loss.item())
            ep_critic += float(critic_loss.item()); ep_ent += float(entropy.item()); ep_batches += 1

            # free small
            del xb, mb, rb, logits, vals, dist, acts, logp, logp_act, reward_t, adv, adv_norm
            gc.collect()

        if ep_batches == 0:
            print(f"  [WARN] no batches processed in epoch {ep}")
            continue

        print(f"[Cluster {cluster_id}] Ep {ep}/{epochs} | loss={ep_loss/ep_batches:.6f} actor={ep_actor/ep_batches:.6f} critic={ep_critic/ep_batches:.6f} ent={ep_ent/ep_batches:.6f}")

        # validation evaluation (deterministic argmax if val exists)
        if X_val is not None:
            model.eval()
            # build val inference in batches
            val_total = X_val.shape[0] * X_val.shape[2]
            val_batchsize = batch_size
            val_rewards = []
            for start in range(0, val_total, val_batchsize):
                xb_list = []; rb_list = []
                for s in range(start, min(val_total, start+val_batchsize)):
                    b, n = divmod(int(s), X_val.shape[2])
                    seq = X_val[b, :, n, :]
                    seq = np.nan_to_num(seq, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
                    xb_list.append(seq)
                    rb_list.append(r_val[b, n])
                if not xb_list:
                    continue
                xb_t = safe_tensor(np.stack(xb_list, axis=0))
                logits_v, vals_v = model(xb_t)
                acts_v = torch.argmax(logits_v, dim=-1)
                rb_t = safe_tensor(np.array(rb_list, dtype=np.float32))
                rdet = torch.where(acts_v==2, rb_t, torch.where(acts_v==0, -rb_t, torch.zeros_like(rb_t)))
                val_rewards.append(rdet.cpu().numpy())
                del xb_t, logits_v, vals_v, acts_v, rb_t, rdet
            if val_rewards:
                val_arr = np.concatenate(val_rewards, axis=0)
                val_score = float(val_arr.mean())
            else:
                val_score = -1e9
            print(f"  [VAL] deterministic mean net reward = {val_score:.6f}")
            if val_score > best_val_score:
                best_val_score = val_score
                torch.save(model.state_dict(), best_path)
                print(f"  ✅ New best checkpoint saved for cluster {cluster_id} (val={val_score:.6f})")

        # always save last
        torch.save(model.state_dict(), last_path)

    # end epochs
    print(f"--- Finished training cluster {cluster_id}. Best val score = {best_val_score:.6f} ---")
    ckpt = best_path if best_path.exists() else last_path
    return str(ckpt)

# =========== Inference (deterministic argmax) ===========
def infer_on_split(output_csv: str, split_name: str = "test"):
    # reset file
    if os.path.exists(output_csv):
        os.remove(output_csv)
    with open(output_csv, "w", newline="") as fh:
        csv.writer(fh).writerow(["date","ticker","signal"])
    for meta in tensor_index_all:
        if meta.get("split") != split_name:
            continue
        cluster_id = int(meta.get("cluster"))
        print(f"[INF] cluster={cluster_id} split={split_name}")
        try:
            X, M, tickers, dates, dates_shifted, split = load_tensor_meta(meta)
        except Exception as e:
            print("  [WARN] load tensor failed:", e); continue
        B, LOOKBACK, N, F = X.shape
        # choose checkpoint: best else last
        best_path = MODEL_DIR / f"a3c_cluster_{cluster_id}_best.pt"
        last_path = MODEL_DIR / f"a3c_cluster_{cluster_id}_last.pt"
        model_path = best_path if best_path.exists() else last_path if last_path.exists() else None
        if model_path is None:
            print(f"  [WARN] no checkpoint for cluster {cluster_id}, skipping")
            continue
        model = A3CNet(n_features=F, hidden=HIDDEN).to(DEVICE)
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        model.eval()
        total = B * N
        with open(output_csv, "a", newline="") as fh:
            w = csv.writer(fh)
            for start in range(0, total, BATCH_SIZE):
                xb_list = []; idx_list = []
                for s in range(start, min(total, start+BATCH_SIZE)):
                    b, n = divmod(int(s), N)
                    seq = X[b, :, n, :]
                    seq = np.nan_to_num(seq, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
                    xb_list.append(seq); idx_list.append((b,n))
                if not xb_list:
                    continue
                xb_t = safe_tensor(np.stack(xb_list, axis=0))
                logits, vals = model(xb_t)
                acts = torch.argmax(logits, dim=-1).cpu().numpy()    # 0,1,2
                acts_mapped = np.where(acts==2, 1, np.where(acts==0, -1, 0))
                for k, (b,n) in enumerate(idx_list):
                    out_date = dates_shifted[b].isoformat() if dates_shifted is not None and len(dates_shifted)>b else pd.to_datetime(dates[b]).date().isoformat()
                    w.writerow([out_date, tickers[n], int(acts_mapped[k])])
                del xb_t, logits, vals, acts
                gc.collect()
        print(f"  [INF] cluster {cluster_id} -> appended signals to {output_csv}")

# =========== MAIN flow ===========
# 1) Train clusters that have split=train entries (iterate over tensor_index_all)
trained = {}
for meta in tensor_index_all:
    if meta.get("split") != "train":
        continue
    try:
        ckpt = train_cluster(meta, epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR)
        if ckpt:
            trained[int(meta.get("cluster"))] = ckpt
    except Exception as e:
        print(f"[ERROR] training cluster {meta.get('cluster')}: {e}")

# 2) After training, run inference on split=test and save CSV
print("\n=== Running inference on split=test and writing to", SIG_INFER_FILE, "===")
infer_on_split(str(SIG_INFER_FILE), split_name="test")
print("\n✅ Block 8 complete: training done and inference saved to", SIG_INFER_FILE)

Device: cpu
df_backtest loaded: dates 2020-01-02 00:00:00 -> 2025-10-02 00:00:00

--- TRAIN cluster 0 (split=train) ---
  windows=193, LOOKBACK=20, N=350, F=16
  Found val windows=230
[Cluster 0] Ep 1/6 | loss=-0.015854 actor=-0.005562 critic=0.000217 ent=1.050907
  [VAL] deterministic mean net reward = 0.000183
  ✅ New best checkpoint saved for cluster 0 (val=0.000183)
[Cluster 0] Ep 2/6 | loss=-0.014637 actor=-0.011404 critic=0.000258 ent=0.349111
  [VAL] deterministic mean net reward = 0.000183
[Cluster 0] Ep 3/6 | loss=-0.008478 actor=-0.007185 critic=0.000267 ent=0.155960
  [VAL] deterministic mean net reward = 0.000183
[Cluster 0] Ep 4/6 | loss=-0.005977 actor=-0.005140 critic=0.000269 ent=0.110622
  [VAL] deterministic mean net reward = 0.000183
[Cluster 0] Ep 5/6 | loss=-0.005695 actor=-0.004631 critic=0.000267 ent=0.133128
  [VAL] deterministic mean net reward = 0.000183
[Cluster 0] Ep 6/6 | loss=-0.005296 actor=-0.003683 critic=0.000261 ent=0.187431
  [VAL] deterministic mean

In [85]:
from pathlib import Path
import os, csv

SIG_DIR = Path("./signals/")
MODEL_DIR = Path("./models_a3c/")   # thêm dòng này cho chắc chắn

SIG_VAL_FILE = SIG_DIR / "a3c_signals_infer_val.csv"

print("\n=== Running inference on split=val and writing to", SIG_VAL_FILE, "===")
infer_on_split(str(SIG_VAL_FILE), split_name="val")
print("\n✅ Block 9b complete: inference saved to", SIG_VAL_FILE)


=== Running inference on split=val and writing to signals\a3c_signals_infer_val.csv ===
[INF] cluster=0 split=val
  [INF] cluster 0 -> appended signals to signals\a3c_signals_infer_val.csv
[INF] cluster=1 split=val
  [INF] cluster 1 -> appended signals to signals\a3c_signals_infer_val.csv
[INF] cluster=2 split=val
  [INF] cluster 2 -> appended signals to signals\a3c_signals_infer_val.csv
[INF] cluster=3 split=val
  [INF] cluster 3 -> appended signals to signals\a3c_signals_infer_val.csv
[INF] cluster=4 split=val
  [INF] cluster 4 -> appended signals to signals\a3c_signals_infer_val.csv
[INF] cluster=5 split=val
  [INF] cluster 5 -> appended signals to signals\a3c_signals_infer_val.csv
[INF] cluster=6 split=val
  [INF] cluster 6 -> appended signals to signals\a3c_signals_infer_val.csv
[INF] cluster=7 split=val
  [INF] cluster 7 -> appended signals to signals\a3c_signals_infer_val.csv
[INF] cluster=8 split=val
  [INF] cluster 8 -> appended signals to signals\a3c_signals_infer_val.csv
[I

**Block 10 : Huấn luyện Cluster DDPG (chỉ với trường hợp vị thế long)**

**Block 10 prep bản có chia train val test**

In [ ]:
# Block 10-prep — Chuẩn bị dữ liệu DDPG (auto-remap clusters, split-aware)
import os, json
import numpy as np
import pandas as pd
from pathlib import Path

# ---------- CONFIG / PATH ----------
BASE_TENSOR_DIR = Path("C:/tensors_out")          
TENSOR_INDEX = BASE_TENSOR_DIR / "tensor_index.json"
SIG_FILE = Path("./signals/a3c_signals_infer.csv") 
DF_BACKTEST = Path("./backtest_ddpg/df_backtest.csv")
OUTPUT_DIR = Path("./ddpg_prep/")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

EXECUTION_LAG = 2        
STATE_LKBK    = 10       
MIN_NAMES_PER_CLUSTER = 5

# ---------- Splits ----------
TRAIN_START = pd.Timestamp("2020-01-01")
TRAIN_END   = pd.Timestamp("2023-12-31")
VAL_START   = pd.Timestamp("2024-01-01")
VAL_END     = pd.Timestamp("2024-12-31")
TEST_START  = pd.Timestamp("2025-01-01")

# ---------- Sanity ----------
if not TENSOR_INDEX.exists():
    raise FileNotFoundError(f"tensor_index.json không tìm thấy tại {TENSOR_INDEX}.")
if not SIG_FILE.exists():
    raise FileNotFoundError(f"Signals file không tìm thấy: {SIG_FILE}.")
if not DF_BACKTEST.exists():
    raise FileNotFoundError(f"Price backtest file không tìm thấy: {DF_BACKTEST}.")

# ---------- Load price ----------
df_px = pd.read_csv(DF_BACKTEST, parse_dates=["date"])
px_wide = df_px.pivot(index="date", columns="ticker", values="close").sort_index()
px_wide = px_wide.ffill().bfill()
TEST_END = px_wide.index.max()
print("Price timeline:", px_wide.index.min(), "→", px_wide.index.max())

# ---------- Load signals ----------
signals = pd.read_csv(SIG_FILE, parse_dates=["date"])
sig_wide_raw = signals.pivot_table(index="date", columns="ticker", values="signal", aggfunc="last").sort_index()
idx_all = px_wide.index.union(sig_wide_raw.index).sort_values()
sig_wide = sig_wide_raw.reindex(idx_all).fillna(0.0)
sig_wide_lag = sig_wide.shift(EXECUTION_LAG).reindex(idx_all).fillna(0.0)

# ---------- Load cluster mapping ----------
with open(TENSOR_INDEX, "r", encoding="utf-8") as fh:
    tensor_index = json.load(fh)

ticker2cluster = {}
for meta in tensor_index:
    for tk in meta.get("tickers", []):
        ticker2cluster[tk] = int(meta.get("cluster", -1))
ticker2cluster = pd.Series(ticker2cluster)

print("Clusters in tensor_index (before remap):", sorted(ticker2cluster.unique().tolist()))

# ---------- Restrict tickers ----------
tickers = sorted(set(px_wide.columns).intersection(ticker2cluster.index))
cluster_of = ticker2cluster.loc[tickers]

# ---------- Remap clusters ----------
active_clusters = sorted(cluster_of.unique().tolist())
remap = {old:i for i, old in enumerate(active_clusters)}
cluster_of = cluster_of.map(remap)

clusters = sorted(remap.values())
cluster_members = {c: cluster_of[cluster_of==c].index.tolist() for c in clusters}

print("Clusters in tensor_index (after remap):", len(clusters), "->", clusters)
print("Tickers used:", len(tickers))

px_wide = px_wide.reindex(columns=tickers)
sig_wide_lag = sig_wide_lag.reindex(columns=tickers).fillna(0.0)
ret_wide = px_wide.pct_change().fillna(0.0)

# ---------- Build state arrays ----------
def build_state_arrays(ret_w, sig_lag_w, start, end, K=STATE_LKBK):
    R = ret_w.loc[start:end]
    S = sig_lag_w.loc[start:end]
    dates = R.index

    act_cols, ret_cols, ACTIVE_masks = [], [], {}
    for c in clusters:
        tks = cluster_members[c]
        if not tks:
            act_c = pd.Series(0.0, index=dates, name=c)
            ret_c = pd.Series(0.0, index=dates, name=c)
            ACTIVE_masks[c] = pd.DataFrame(0.0, index=dates, columns=[])
        else:
            S_c, R_c = S[tks], R[tks]
            active_mask = (S_c != 0).astype("float32")
            ACTIVE_masks[c] = active_mask.copy()
            denom = active_mask.sum(axis=1).replace(0, np.nan)
            w = active_mask.div(denom, axis=0).fillna(0.0)
            act_c = active_mask.mean(axis=1).astype("float32")
            ret_c = (R_c * w).sum(axis=1).astype("float32")
        act_cols.append(act_c.rename(c))
        ret_cols.append(ret_c.rename(c))

    act_df = pd.concat(act_cols, axis=1).astype("float32")
    cret_df = pd.concat(ret_cols, axis=1).astype("float32")

    def stack_lookback(df, K):
        mats = []
        for k in range(K):
            mats.append(df.shift(k).fillna(0.0))
        arr = np.stack([m.values for m in mats], axis=2)  
        return arr

    A3, R3 = stack_lookback(act_df, K), stack_lookback(cret_df, K)
    valid_mask = np.arange(A3.shape[0]) >= (K - 1)
    dates2 = dates[valid_mask]

    A3_valid, R3_valid = A3[valid_mask], R3[valid_mask]
    Tv = A3_valid.shape[0]
    S_mat = np.concatenate([A3_valid.reshape(Tv,-1), R3_valid.reshape(Tv,-1)], axis=1).astype("float32")
    R_mat = cret_df.loc[dates2].values.astype("float32")
    ACTIVE_trim = {c: ACTIVE_masks[c].loc[dates2] for c in clusters}
    return S_mat, R_mat, dates2, ACTIVE_trim

print("Building state matrices...")
S_train, R_train, d_train, _ = build_state_arrays(ret_wide, sig_wide_lag, TRAIN_START, TRAIN_END)
S_val,   R_val,   d_val,   _ = build_state_arrays(ret_wide, sig_wide_lag, VAL_START, VAL_END)
S_test,  R_test,  d_test,  _ = build_state_arrays(ret_wide, sig_wide_lag, TEST_START, TEST_END)

print("Shapes:")
print(" S_train:", S_train.shape, " R_train:", R_train.shape)
print(" S_val:  ", S_val.shape,   " R_val:  ", R_val.shape)
print(" S_test: ", S_test.shape,  " R_test: ", R_test.shape)

# ---------- Save ----------
np.save(OUTPUT_DIR/"S_train.npy", S_train)
np.save(OUTPUT_DIR/"R_train.npy", R_train)
np.save(OUTPUT_DIR/"S_val.npy", S_val)
np.save(OUTPUT_DIR/"R_val.npy", R_val)
np.save(OUTPUT_DIR/"S_test.npy", S_test)
np.save(OUTPUT_DIR/"R_test.npy", R_test)

pd.Series(d_train.astype(str)).to_csv(OUTPUT_DIR/"dates_train.csv", index=False)
pd.Series(d_val.astype(str)).to_csv(OUTPUT_DIR/"dates_val.csv", index=False)
pd.Series(d_test.astype(str)).to_csv(OUTPUT_DIR/"dates_test.csv", index=False)

pd.Series(clusters).to_csv(OUTPUT_DIR/"clusters_list.csv", index=False)
with open(OUTPUT_DIR/"cluster_members.json","w",encoding="utf-8") as fh:
    json.dump({int(c): cluster_members[c] for c in clusters}, fh, indent=2, ensure_ascii=False)

print("✅ Block 10-prep done. Files in", OUTPUT_DIR)

Price timeline: 2020-01-02 00:00:00 → 2025-10-02 00:00:00
Clusters in tensor_index (before remap): [5, 8, 9]
Clusters in tensor_index (after remap): 3 -> [0, 1, 2]
Tickers used: 391
Building state matrices...
Shapes:
 S_train: (991, 60)  R_train: (991, 3)
 S_val:   (241, 60)  R_val:   (241, 3)
 S_test:  (176, 60)  R_test:  (176, 3)
✅ Block 10-prep done. Files in ddpg_prep


In [76]:
# Đếm số mã mỗi cụm (sau khi giao cắt tickers)
print(cluster_of.value_counts().sort_index())

5      1
8     34
9    356
Name: count, dtype: int64


**Block 10a mới bản có chia ra train val test**

In [1]:
# Block 10a — Train DDPG-like actor–critic (fixed & stable)
import os, gc, math, time
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

# ---------- PATHS ----------
PREP_DIR   = Path("./ddpg_prep/")
MODEL_DIR  = Path("./ddpg_models/")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# ---------- HYPER ----------
EPOCHS     = 60
BATCH_SIZE = 64
LR_ACTOR   = 1e-4
LR_CRITIC  = 5e-4
GAMMA      = 0.95
TAU        = 1e-2
NOISE_STD  = 0.05
HIDDEN     = 128
SEED       = 42
REWARD_NORM = True    # normalize rewards (per-batch)
CLIP_GRAD = 1.0

np.random.seed(SEED); torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------- Load prepared data ----------
S_train = np.load(PREP_DIR / "S_train.npy")    # (T, s_dim)
R_train = np.load(PREP_DIR / "R_train.npy")    # (T, a_dim)
dates_train = pd.read_csv(PREP_DIR / "dates_train.csv", header=None).squeeze().astype(str).values

# sanity
assert S_train.ndim == 2, "S_train should be 2D (T, s_dim)"
assert R_train.ndim == 2, "R_train should be 2D (T, a_dim)"
T_total, s_dim = S_train.shape
_, a_dim = R_train.shape
print("Train shapes:", S_train.shape, R_train.shape)

# cast
S_train = S_train.astype(np.float32)
R_train = R_train.astype(np.float32)

# optional: standardize states (fit on train)
scaler = StandardScaler()
S_train = scaler.fit_transform(S_train).astype(np.float32)

# ---------- Networks ----------
class Actor(nn.Module):
    def __init__(self, s_dim, a_dim, hidden=HIDDEN):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(s_dim, hidden), nn.ReLU(),
            nn.Linear(hidden, hidden), nn.ReLU(),
            nn.Linear(hidden, a_dim)
        )
    def forward(self, s):
        # return logits (pre-softmax) to allow stable noise injection outside
        return self.net(s)

class Critic(nn.Module):
    def __init__(self, s_dim, a_dim, hidden=HIDDEN):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(s_dim + a_dim, hidden), nn.ReLU(),
            nn.Linear(hidden, hidden), nn.ReLU(),
            nn.Linear(hidden, 1)
        )
    def forward(self, s, a):
        x = torch.cat([s, a], dim=-1)
        return self.net(x).squeeze(-1)  # return (batch,)

# ---------- Replay Buffer ----------
class Buffer:
    def __init__(self, maxlen=20000):
        self.buf=[]; self.maxlen=maxlen
    def push(self, s,a,r,s2):
        if len(self.buf)>=self.maxlen: self.buf.pop(0)
        self.buf.append((s.astype(np.float32), a.astype(np.float32), float(r), s2.astype(np.float32)))
    def __len__(self):
        return len(self.buf)
    def sample(self, bs):
        n = min(bs, len(self.buf))
        idx = np.random.choice(len(self.buf), n, replace=False)
        s,a,r,s2 = zip(*[self.buf[i] for i in idx])
        return (np.stack(s), np.stack(a), np.array(r, dtype=np.float32).reshape(-1,1), np.stack(s2))

def soft_update(src, tgt, tau):
    with torch.no_grad():
        for p, tp in zip(src.parameters(), tgt.parameters()):
            tp.data.mul_(1.0 - tau)
            tp.data.add_(tau * p.data)

def port_reward(w, r):
    # w: (a_dim,) weights (sum=1), r: (a_dim,) returns for clusters; return scalar PnL
    return float(np.dot(w, r))

# ---------- Init models & optim ----------
actor = Actor(s_dim, a_dim).to(device)
critic = Critic(s_dim, a_dim).to(device)
t_actor = Actor(s_dim, a_dim).to(device); t_actor.load_state_dict(actor.state_dict())
t_critic = Critic(s_dim, a_dim).to(device); t_critic.load_state_dict(critic.state_dict())

optA = optim.Adam(actor.parameters(), lr=LR_ACTOR)
optC = optim.Adam(critic.parameters(), lr=LR_CRITIC)
mse = nn.MSELoss()

buf = Buffer(maxlen=50000)

# ---------- Training loop ----------
print("Starting training...")
start_time = time.time()
for ep in range(1, EPOCHS+1):
    epoch_c_loss = 0.0
    epoch_a_loss = 0.0
    n_updates = 0

    # iterate temporally: each step t -> transition (S[t], w_t, r_t, S[t+1])
    for t in range(0, T_total - 1):
        s = S_train[t]      # (s_dim,)
        s2 = S_train[t+1]
        # actor deterministic output (logits), then add noise in logit space
        actor.eval()
        with torch.no_grad():
            logits = actor(torch.from_numpy(s).float().unsqueeze(0).to(device)).cpu().numpy().squeeze(0)  # (a_dim,)
        # safe log and noise
        EPS = 1e-9
        logits = np.clip(logits, -1e6, 1e6)   # numeric guard
        # add gaussian noise in logit space
        noisy = logits + np.random.normal(0, NOISE_STD, size=logits.shape)
        # compute weights via softmax of noisy logits
        expv = np.exp(noisy - noisy.max())
        w_e = (expv / (expv.sum() + EPS)).astype(np.float32)

        # immediate portfolio reward using cluster returns at t
        r_scalar = port_reward(w_e, R_train[t])
        buf.push(s, w_e, r_scalar, s2)

        # update if buffer has enough
        if len(buf) >= BATCH_SIZE:
            sb, ab, rb, s2b = buf.sample(BATCH_SIZE)
            sb_t = torch.from_numpy(sb).float().to(device)       # (B, s_dim)
            ab_t = torch.from_numpy(ab).float().to(device)       # (B, a_dim)
            rb_t = torch.from_numpy(rb).float().squeeze(-1).to(device)  # (B,)

            s2b_t = torch.from_numpy(s2b).float().to(device)

            # normalize rewards per-batch if desired
            if REWARD_NORM:
                rb_mean = rb_t.mean()
                rb_std = rb_t.std(unbiased=False) + 1e-8
                rb_t = (rb_t - rb_mean) / rb_std

            # Critic target: y = r + gamma * Q_target(s2, a2)
            with torch.no_grad():
                a2_logits = t_actor(s2b_t)  # logits
                a2_exp = torch.exp(a2_logits - a2_logits.max(dim=1, keepdim=True)[0])
                a2 = a2_exp / (a2_exp.sum(dim=1, keepdim=True) + 1e-9)  # (B, a_dim)
                q2 = t_critic(s2b_t, a2)   # (B,)
                y = rb_t + GAMMA * q2

            # Critic update
            q = critic(sb_t, ab_t)
            lc = mse(q, y)
            optC.zero_grad(); lc.backward(); torch.nn.utils.clip_grad_norm_(critic.parameters(), CLIP_GRAD); optC.step()

            # Actor update (maximize value predicted by critic)
            ap_logits = actor(sb_t)
            ap_exp = torch.exp(ap_logits - ap_logits.max(dim=1, keepdim=True)[0])
            ap = ap_exp / (ap_exp.sum(dim=1, keepdim=True) + 1e-9)
            la = - critic(sb_t, ap).mean()
            optA.zero_grad(); la.backward(); torch.nn.utils.clip_grad_norm_(actor.parameters(), CLIP_GRAD); optA.step()

            # soft update targets
            soft_update(actor, t_actor, TAU)
            soft_update(critic, t_critic, TAU)

            epoch_c_loss += float(lc.item())
            epoch_a_loss += float(la.item())
            n_updates += 1

            # free
            del sb_t, ab_t, rb_t, s2b_t, a2_logits, a2_exp, a2, q2, y, q, lc, ap_logits, ap, la
            torch.cuda.empty_cache()

    # epoch logging
    if n_updates > 0:
        print(f"[Epoch {ep}/{EPOCHS}] Critic={epoch_c_loss / n_updates:.6f} | Actor={epoch_a_loss / n_updates:.6f} | updates={n_updates}")
    else:
        print(f"[Epoch {ep}/{EPOCHS}] no updates (buffer too small)")

    # periodic checkpoint
    if ep % 10 == 0 or ep == EPOCHS:
        torch.save(actor.state_dict(), MODEL_DIR / f"actor_ep{ep}.pt")
        torch.save(critic.state_dict(), MODEL_DIR / f"critic_ep{ep}.pt")
        print("  -> Saved checkpoint at epoch", ep)

    gc.collect()
    torch.cuda.empty_cache()

# final save
torch.save(actor.state_dict(), MODEL_DIR / "actor_final.pt")
torch.save(critic.state_dict(), MODEL_DIR / "critic_final.pt")
print("✅ Block 10a done. Models saved to", MODEL_DIR, "Elapsed:", time.time() - start_time)

Device: cpu
Train shapes: (991, 60) (991, 3)
Starting training...
[Epoch 1/60] Critic=0.000000 | Actor=0.038839 | updates=927
[Epoch 2/60] Critic=0.000000 | Actor=0.023845 | updates=990
[Epoch 3/60] Critic=0.000000 | Actor=0.014240 | updates=990
[Epoch 4/60] Critic=0.000000 | Actor=0.008509 | updates=990
[Epoch 5/60] Critic=0.000000 | Actor=0.004987 | updates=990
[Epoch 6/60] Critic=0.000000 | Actor=0.002926 | updates=990
[Epoch 7/60] Critic=0.000000 | Actor=0.001760 | updates=990
[Epoch 8/60] Critic=0.000000 | Actor=0.000930 | updates=990
[Epoch 9/60] Critic=0.000000 | Actor=0.000314 | updates=990
[Epoch 10/60] Critic=0.000000 | Actor=0.000017 | updates=990
  -> Saved checkpoint at epoch 10
[Epoch 11/60] Critic=0.000000 | Actor=-0.000455 | updates=990
[Epoch 12/60] Critic=0.000000 | Actor=-0.000460 | updates=990
[Epoch 13/60] Critic=0.000000 | Actor=-0.000368 | updates=990
[Epoch 14/60] Critic=0.000000 | Actor=-0.000297 | updates=990
[Epoch 15/60] Critic=0.000000 | Actor=-0.000223 | u

**code mới nhất nằm ngày trên**

**BLock 10B bản có chia ra val train test**

In [ ]:
# Block 10B — Backtest (VAL-only or TEST) with trade_log, holdings and telegram payload
import os, gc, json, csv, math, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import datetime

warnings.filterwarnings("ignore")

# ----------------- CONFIG -----------------
BASE_OUT      = Path("./backtest_grid_val/")   # output root
BASE_OUT.mkdir(parents=True, exist_ok=True)

DATA_DIR      = "./tensors/"
SIG_DIR       = "./signals/"
MODEL_DIR     = "./ddpg_models/"
OUTPUT_ROOT   = BASE_OUT                        # will create per-config subfolders

SIG_FILE      = os.path.join(SIG_DIR, "a3c_signals_infer_val.csv")   # ensure this contains VAL signals
DF_PRICE      = "./backtest_ddpg/df_backtest.csv"                   # OHLC price (preferred)
FALLBACK_PRICE= "vnindex_price_fa_merged.csv"
ACTOR_PATH    = os.path.join(MODEL_DIR, "actor_final.pt")           # actor file (not used here but sanity)
TENSOR_INDEX  = os.path.join("C:/tensors_out", "tensor_index.json")  # mapping

GRID_SL       = [0.01, 0.02, 0.03, 0.05]
GRID_TP       = [0.02, 0.04, 0.06, 0.10]

ENTRY_LAG     = 2     # signal T -> entry at open T+ENTRY_LAG
INIT_CAP      = 10_000.0
TOP_K_PER_CLUSTER = 5
MIN_NAMES_PER_CLUSTER = 1
MAX_HOLD_DAYS = 30
COST_BPS      = 30   # per turnover
STATE_LKBK    = 10

# VAL range (no leakage); set to desired validation range
VAL_START = pd.Timestamp("2024-01-01")
VAL_END   = pd.Timestamp("2024-12-31")
VAL_ONLY = True

# ----------------- Helper funcs -----------------
def intraday_check(entry_price, day_high, day_low, sl_pct, tp_pct):
    if pd.isna(entry_price) or pd.isna(day_high) or pd.isna(day_low):
        return None, "hold"
    sl_price = entry_price * (1 - sl_pct)
    tp_price = entry_price * (1 + tp_pct)
    # conservative: treat SL as hit first if both satisfy (user choice)
    if day_low <= sl_price and day_high >= tp_price:
        # which first? choose SL-first conservative
        return sl_price, "sl"
    if day_low <= sl_price:
        return sl_price, "sl"
    if day_high >= tp_price:
        return tp_price, "tp"
    return None, "hold"

def compute_stats_from_series(port_series):
    port_series = port_series.dropna()
    if len(port_series) < 2:
        return {"ROI":0.0,"Vol":0.0,"Sharpe":0.0,"MaxDD":0.0,"WinRate":0.0}
    ret = port_series.pct_change().dropna()
    roi = port_series.iloc[-1]/port_series.iloc[0]-1.0
    vol = ret.std() * math.sqrt(252) if len(ret)>1 else 0.0
    sharpe = (ret.mean()/ret.std()) * math.sqrt(252) if ret.std()>0 else 0.0
    dd = (port_series/port_series.cummax()-1.0).min()
    winrate = float((ret>0).mean())
    return {"ROI":roi, "Vol":vol, "Sharpe":sharpe, "MaxDD":dd, "WinRate":winrate}

# ----------------- Load price -----------------
if not os.path.exists(DF_PRICE):
    print(f"Warning: {DF_PRICE} not found, trying fallback {FALLBACK_PRICE}")
    if not os.path.exists(FALLBACK_PRICE):
        raise FileNotFoundError(f"Neither {DF_PRICE} nor fallback {FALLBACK_PRICE} found.")
    df_px = pd.read_csv(FALLBACK_PRICE)
else:
    df_px = pd.read_csv(DF_PRICE)

# normalize date col
if "timestamp" in df_px.columns:
    df_px = df_px.rename(columns={"timestamp":"date"})
if "date" not in df_px.columns:
    raise ValueError("Price data needs 'date' or 'timestamp' column")

df_px["date"] = pd.to_datetime(df_px["date"])

# require OHLC and ticker
for c in ["open","high","low","close","ticker","date"]:
    if c not in df_px.columns:
        raise ValueError(f"Price data missing column '{c}'. Found: {df_px.columns.tolist()}")

# keep only VAL period (prevent leakage)
df_px = df_px[(df_px["date"] >= VAL_START) & (df_px["date"] <= VAL_END)].copy()
if df_px.empty:
    raise ValueError("No price rows inside VAL range. Check DF_PRICE and VAL_START/VAL_END")

# pivot to wide OHLC
px_open  = df_px.pivot(index="date", columns="ticker", values="open").sort_index()
px_high  = df_px.pivot(index="date", columns="ticker", values="high").sort_index()
px_low   = df_px.pivot(index="date", columns="ticker", values="low").sort_index()
px_close = df_px.pivot(index="date", columns="ticker", values="close").sort_index()

px_index = px_close.index.sort_values().unique()
if len(px_index) == 0:
    raise ValueError("No trading dates after pivot.")

# ----------------- Load signals & mapping -----------------
if not os.path.exists(SIG_FILE):
    raise FileNotFoundError(f"Signals file not found: {SIG_FILE}")
signals = pd.read_csv(SIG_FILE)
if "date" in signals.columns:
    signals["date"] = pd.to_datetime(signals["date"])
elif "timestamp" in signals.columns:
    signals["date"] = pd.to_datetime(signals["timestamp"])
else:
    raise ValueError("Signals csv must contain 'date' or 'timestamp' column")
# restrict signals to VAL range
signals = signals[(signals["date"] >= VAL_START) & (signals["date"] <= VAL_END)].copy()
if signals.empty:
    raise FileNotFoundError("No signals found in VAL range. Run inference for VAL first (Block 9b).")

if not os.path.exists(TENSOR_INDEX):
    raise FileNotFoundError("tensor_index.json not found.")
with open(TENSOR_INDEX,"r",encoding="utf-8") as fh:
    tensor_index = json.load(fh)

ticker2cluster = {}
for meta in tensor_index:
    c = int(meta.get("cluster", -1))
    for tk in meta.get("tickers", []):
        if tk not in ticker2cluster:
            ticker2cluster[tk] = c
ticker2cluster = pd.Series(ticker2cluster)

# intersection of tickers present in price and mapping
tickers = sorted(list(set(px_close.columns).intersection(set(ticker2cluster.index))))
if len(tickers) == 0:
    raise ValueError("No tickers overlap between price and cluster mapping (within VAL).")

px_open  = px_open.reindex(columns=tickers)
px_high  = px_high.reindex(columns=tickers)
px_low   = px_low.reindex(columns=tickers)
px_close = px_close.reindex(columns=tickers)

# signals -> wide (last per day) and align to px_index
sig_wide = signals.pivot_table(index="date", columns="ticker", values="signal", aggfunc="last")
sig_wide = sig_wide.reindex(px_index).reindex(columns=tickers).fillna(0.0)

# shift for execution lag
sig_entry = sig_wide.shift(ENTRY_LAG).reindex(px_index).fillna(0.0)

# clusters
cluster_of = ticker2cluster.loc[tickers]
clusters = sorted(cluster_of.unique().tolist())
cluster_members = {c: cluster_of[cluster_of==c].index.tolist() for c in clusters}
C = len(clusters)
print(f"Tickers: {len(tickers)} | Clusters in-use: {len(clusters)}")

# ----------------- Build safe valid_index (dates with next-day inside VAL) -----------------
# For safety with next-day returns we require the next trading date to exist (we use close/prev_close).
all_dates = list(px_index)
valid_dates = []
for i, d in enumerate(all_dates):
    # we will need next day to compute prev_close for next day returns; but our logic uses same-day open->close and close/prev_close
    if i+1 < len(all_dates):
        # ensure next exists inside VAL (it is because px_index derived from VAL)
        valid_dates.append(d)
valid_index = pd.DatetimeIndex(valid_dates)
if len(valid_index) == 0:
    raise ValueError("No valid dates for backtest after geting valid_index.")

# reindex to valid_index
px_open  = px_open.reindex(valid_index)
px_high  = px_high.reindex(valid_index)
px_low   = px_low.reindex(valid_index)
px_close = px_close.reindex(valid_index)
sig_entry = sig_entry.reindex(valid_index).fillna(0.0)

# compute daily returns used at cluster-level (close/prev_close)
ret_wide = px_close.pct_change().fillna(0.0)

# ----------------- Build S_all/R_all (cluster-level state arrays) -----------------
# We only need S_all to run actor inference — but for backtest we already precomputed actor weights externally.
# Keep minimal: we will not run actor here; we assume cluster_weights will be provided or actor is available.
# To keep compatibility, attempt to load precomputed cluster_weights file if exists (ddpg_prep),
# else fall back to uniform cluster weights.
PRECOMP_CLUSTER_WEIGHTS = None  # path if you have cluster weights precomputed per date (optional)

# fallback function to compute cluster-level stats (act fraction and cluster return)
def cluster_level_from_signals(sig_entry_df, ret_df, clusters, cluster_members, K=STATE_LKBK):
    """Return cluster-level A3 (activity fraction) and cret (cluster returns) per date (T x C)"""
    dates = ret_df.index
    act_cols, ret_cols = [], []
    for c in clusters:
        tks = cluster_members.get(c, [])
        if len(tks)==0:
            act_c = pd.Series(0.0, index=dates, name=c)
            ret_c = pd.Series(0.0, index=dates, name=c)
        else:
            S_c = sig_entry_df.reindex(columns=tks).fillna(0.0)
            R_c = ret_df.reindex(columns=tks).fillna(0.0)
            active_mask = (S_c > 0).astype("float32")
            denom = active_mask.sum(axis=1).replace(0, np.nan)
            w = active_mask.div(denom, axis=0).fillna(0.0)
            act_c = active_mask.mean(axis=1).astype("float32")
            ret_c = (R_c * w).sum(axis=1).astype("float32")
        act_cols.append(act_c.rename(c))
        ret_cols.append(ret_c.rename(c))
    act_df = pd.concat(act_cols, axis=1).astype("float32")
    cret_df = pd.concat(ret_cols, axis=1).astype("float32")
    return act_df, cret_df

# compute simple cluster_weights per date (uniform on active clusters) if actor not available
# Here we try to load actor-based weights computed previously (S_all->actor). If not found we fallback uniform by cluster active fraction.
cluster_weights_by_date = None
if PRECOMP_CLUSTER_WEIGHTS and os.path.exists(PRECOMP_CLUSTER_WEIGHTS):
    cluster_weights_by_date = pd.read_csv(PRECOMP_CLUSTER_WEIGHTS, parse_dates=["date"], index_col=0)
    # shape must be (dates x C)
    cluster_weights = cluster_weights_by_date.values
else:
    # fallback: uniform weights across clusters proportional to act fraction (or uniform)
    act_df, cret_df = cluster_level_from_signals(sig_entry, ret_wide, clusters, cluster_members, K=STATE_LKBK)
    # avoid all-zero
    act_sum = act_df.sum(axis=1)
    cluster_weights = []
    for idx, row in act_df.iterrows():
        if row.sum() <= 1e-9:
            w = np.ones(len(clusters), dtype="float32") / len(clusters)
        else:
            # weight clusters proportionally to fraction active, but ensure sum=1
            w = row.values.astype("float32")
            s = w.sum()
            if s <= 0: w = np.ones_like(w) / len(w)
            else: w = w / s
        cluster_weights.append(w)
    cluster_weights = np.stack(cluster_weights, axis=0)

# ----------------- Grid search loop (per SL/TP) -----------------
summary_rows = []

def run_config(sl, tp):
    cfg_name = f"VAL_SL{int(sl*100)}_TP{int(tp*100)}"
    out_dir = OUTPUT_ROOT / cfg_name
    out_dir.mkdir(parents=True, exist_ok=True)
    print(f"\n=== {cfg_name} ===")

    dates = list(valid_index)
    n_days = len(dates)
    capital = float(INIT_CAP)
    port_vals = pd.Series(index=dates, dtype="float64")
    if n_days>0: port_vals.iloc[0] = capital

    prev_weights = pd.Series(0.0, index=tickers)      # prev target ticker weights
    pos_info = {}   # ticker -> {entry_price, entry_date, weight, hold_days, shares, notional}
    trade_log = []
    holding_log = []
    turnover_series = pd.Series(index=dates, dtype="float64")
    daily_returns = pd.Series(index=dates, dtype="float64")

    # precomputed cluster weights aligned to dates length
    cw = cluster_weights.copy()
    if cw.shape[0] != len(dates):
        # try align by index length; else repeat last
        if cw.shape[0] > len(dates):
            cw = cw[:len(dates)]
        else:
            last = cw[-1]
            extra = np.tile(last, (len(dates)-cw.shape[0],1))
            cw = np.vstack([cw, extra])

    for i, dt in enumerate(dates):
        capital_before = capital
        w_c = cw[i]   # (C,)

        # build target ticker weights from cluster weights and active candidates
        target_w_ticker = pd.Series(0.0, index=tickers, dtype="float32")
        for j, c in enumerate(clusters):
            members = cluster_members.get(c, [])
            if not members: continue
            # candidates where sig_entry>0 on this date
            try:
                row_sig = sig_entry.loc[dt]
            except KeyError:
                row_sig = pd.Series(0.0, index=tickers)
            candidates = [tk for tk in members if (tk in sig_entry.columns and float(sig_entry.loc[dt, tk])>0)]
            if len(candidates) < MIN_NAMES_PER_CLUSTER:
                continue
            # simple momentum ranking (past 5 days within valid_index)
            mom_scores = {}
            pos_dt = i
            start_idx = max(0, pos_dt - 5)
            dstart = dates[start_idx]
            for tk in candidates:
                try:
                    vstart = px_close.loc[dstart, tk]
                    vend = px_close.loc[dt, tk]
                    r = (vend / vstart - 1.0) if (not pd.isna(vstart) and vstart>0) else 0.0
                except Exception:
                    r = 0.0
                mom_scores[tk] = r
            topk = sorted(mom_scores.keys(), key=lambda x: mom_scores[x], reverse=True)[:TOP_K_PER_CLUSTER]
            if len(topk)==0: continue
            share = float(w_c[j]) / len(topk)
            for tk in topk:
                target_w_ticker[tk] += share

        # normalize and zero-out tickers with no price today
        valid_price_mask = (~px_close.loc[dt].isna()).astype(float)
        valid_tks = [tk for tk in tickers if valid_price_mask.get(tk, 0.0) > 0.0]
        ssum = float(target_w_ticker.sum())
        if ssum <= 1e-12:
            # fallback uniform across available tickers
            cnt = max(1, len(valid_tks))
            target_w_ticker[:] = 0.0
            for tk in valid_tks:
                target_w_ticker[tk] = 1.0 / cnt
        else:
            # zero out missing price tickers
            for tk in tickers:
                if tk not in valid_tks:
                    target_w_ticker[tk] = 0.0
            s2 = float(target_w_ticker.sum())
            if s2 <= 1e-12:
                cnt = max(1, len(valid_tks))
                for tk in tickers:
                    target_w_ticker[tk] = (1.0/cnt) if tk in valid_tks else 0.0
            else:
                target_w_ticker = target_w_ticker / float(s2)

        turnover = float(np.sum(np.abs(target_w_ticker.values - prev_weights.values)))
        turnover_series.iloc[i] = turnover

        # 1) Intraday exits (SL/TP) for currently held positions - BEFORE new entries (as requested)
        realized_pnl_currency = 0.0
        closed_tks = []
        for tk, info in list(pos_info.items()):
            entry_price = info["entry_price"]
            shares = info.get("shares", 0.0)
            day_high = px_high.loc[dt, tk] if (tk in px_high.columns and dt in px_high.index) else np.nan
            day_low  = px_low.loc[dt, tk] if (tk in px_low.columns and dt in px_low.index) else np.nan
            exit_price, exit_type = intraday_check(entry_price, day_high, day_low, sl, tp)
            if exit_price is not None:
                # compute realized pnl in currency (based on shares)
                realized_pnl_curr = shares * (exit_price - entry_price)
                realized_pnl_currency += float(realized_pnl_curr)
                # log SELL
                trade_log.append({
                    "date": pd.Timestamp(dt).isoformat(),
                    "ticker": tk,
                    "action": "SELL",
                    "price": float(exit_price),
                    "weight": float(info["weight"]),
                    "shares": float(shares),
                    "notional_at_entry": float(info["notional"]),
                    "pnl_currency": float(realized_pnl_curr),
                    "reason": exit_type
                })
                closed_tks.append(tk)
                del pos_info[tk]

        # 2) New entries / rebalances at open (after exits)
        new_entries = []
        for tk in tickers:
            new_w = float(target_w_ticker[tk])
            old_w = float(prev_weights.get(tk, 0.0))
            # if target weight >0 and change relative to position -> create/update
            if new_w > 0 and (tk not in pos_info or abs(new_w - old_w) > 1e-6):
                op = px_open.loc[dt, tk] if (tk in px_open.columns and dt in px_open.index) else np.nan
                if pd.isna(op) or op == 0:
                    continue
                notional = capital_before * new_w
                shares = (notional / op) if op>0 else 0.0
                pos_info[tk] = {"entry_price": float(op), "entry_date": dt, "weight": new_w, "hold_days": 0, "shares": float(shares), "notional": float(notional)}
                new_entries.append(tk)
                trade_log.append({
                    "date": pd.Timestamp(dt).isoformat(),
                    "ticker": tk,
                    "action": "BUY",
                    "price": float(op),
                    "weight": float(new_w),
                    "shares": float(shares),
                    "notional_at_entry": float(notional),
                    "pnl_currency": 0.0,
                    "reason": "entry/rebalance"
                })
            else:
                # weight update for existing positions
                if tk in pos_info:
                    pos_info[tk]["weight"] = new_w

        prev_weights = target_w_ticker.copy()

        # 3) Compute day returns (based on current positions after exits and after new entries executed at open)
        day_ret_fraction = 0.0
        prev_close = px_close.shift(1).loc[dt] if dt in px_close.index else pd.Series(index=tickers, data=[np.nan]*len(tickers))
        close_today = px_close.loc[dt]

        for tk in tickers:
            w = float(prev_weights[tk])
            if math.isclose(w, 0.0):
                continue
            if tk in new_entries:
                # new entries: open -> close
                op = px_open.loc[dt, tk] if (tk in px_open.columns and dt in px_open.index) else np.nan
                cl = close_today.get(tk, np.nan)
                if pd.isna(op) or pd.isna(cl) or op==0:
                    ret = 0.0
                else:
                    ret = (cl / op - 1.0)
            else:
                prev_c = prev_close.get(tk, np.nan)
                cl = close_today.get(tk, np.nan)
                if pd.isna(prev_c) or prev_c==0 or pd.isna(cl):
                    ret = 0.0
                else:
                    ret = (cl / prev_c - 1.0)
            day_ret_fraction += w * float(ret)

        # realized_pnl_currency from SL/TP already computed; convert to fraction of capital_before
        realized_pnl_fraction = (realized_pnl_currency / capital_before) if capital_before != 0 else 0.0

        total_ret = day_ret_fraction + realized_pnl_fraction
        fee = (COST_BPS / 1e4) * turnover
        total_ret_net = total_ret - fee

        # update capital multiplicatively
        capital = capital_before * (1.0 + total_ret_net)
        port_vals.iloc[i] = capital
        daily_returns.iloc[i] = total_ret_net

        # log holdings snapshot at end of day (after update)
        for tk, info in pos_info.items():
            holding_log.append({
                "date": pd.Timestamp(dt).isoformat(),
                "ticker": tk,
                "weight": float(info["weight"]),
                "entry_price": float(info["entry_price"]),
                "shares": float(info.get("shares", 0.0)),
                "hold_days": int(info.get("hold_days", 0)),
                "notional_at_entry": float(info.get("notional", 0.0))
            })

        # increment hold_days and forced close if too long
        for tk in list(pos_info.keys()):
            pos_info[tk]["hold_days"] = pos_info[tk].get("hold_days", 0) + 1
            if pos_info[tk]["hold_days"] >= MAX_HOLD_DAYS:
                # forced close at close price today (mark-to-market)
                cl = px_close.loc[dt, tk] if (tk in px_close.columns and dt in px_close.index) else np.nan
                if not pd.isna(cl):
                    shares = pos_info[tk].get("shares", 0.0)
                    entry_price = pos_info[tk]["entry_price"]
                    realized_pnl_curr = shares * (cl - entry_price)
                    trade_log.append({
                        "date": pd.Timestamp(dt).isoformat(),
                        "ticker": tk,
                        "action": "SELL",
                        "price": float(cl),
                        "weight": float(pos_info[tk]["weight"]),
                        "shares": float(shares),
                        "notional_at_entry": float(pos_info[tk].get("notional", 0.0)),
                        "pnl_currency": float(realized_pnl_curr),
                        "reason": "forced_hold_limit"
                    })
                del pos_info[tk]

        # update prev_weights already done
        # housekeeping
        gc.collect()
    # end dates loop

    # Save outputs
    pd.Series(port_vals).to_csv(out_dir / "portfolio_value.csv")
    pd.Series(daily_returns).to_csv(out_dir / "daily_returns.csv")
    turnover_series.to_csv(out_dir / "turnover.csv")

    trade_df = pd.DataFrame(trade_log)
    holding_df = pd.DataFrame(holding_log)
    trade_df.to_csv(out_dir / "trade_log.csv", index=False)
    holding_df.to_csv(out_dir / "holdings_snapshot.csv", index=False)

    # telegram payload + latest holdings
    final_cap = float(port_vals.dropna().iloc[-1]) if port_vals.dropna().size>0 else float(capital)
    stats = compute_stats_from_series(port_vals)
    top_holdings = holding_df[holding_df["date"]==holding_df["date"].max()].sort_values("weight", ascending=False).head(10) if not holding_df.empty else pd.DataFrame()
    telegram_text = []
    telegram_text.append(f"Config: {cfg_name}")
    telegram_text.append(f"Final capital: {final_cap:,.2f}")
    telegram_text.append(f"ROI: {stats['ROI']:.2%} | Sharpe: {stats['Sharpe']:.3f} | MaxDD: {stats['MaxDD']:.2%}")
    telegram_text.append("Top holdings (end):")
    if not top_holdings.empty:
        for _, r in top_holdings.iterrows():
            telegram_text.append(f"  {r['ticker']}: weight={r['weight']:.2%}, shares={int(r['shares'])}, entry={r['entry_price']:.2f}")
    else:
        telegram_text.append("  (no holdings)")
    with open(out_dir / "telegram_message.txt", "w", encoding="utf-8") as fh:
        fh.write("\n".join(telegram_text))

    # also save json payload with summary + last holdings for easy reading by block12
    payload = {"config": cfg_name, "final_capital": final_cap, "stats": stats}
    payload["latest_holdings"] = top_holdings.to_dict(orient="records") if not top_holdings.empty else []
    with open(out_dir / "telegram_payload.json", "w", encoding="utf-8") as fh:
        json.dump(payload, fh, indent=2, ensure_ascii=False)

    # return summary row
    res = {
        "config": cfg_name,
        "sl": sl,
        "tp": tp,
        "ROI": stats["ROI"],
        "Vol": stats["Vol"],
        "Sharpe": stats["Sharpe"],
        "MaxDD": stats["MaxDD"],
        "WinRate": stats["WinRate"],
        "final_capital": final_cap,
        "n_days": len(port_vals.dropna())
    }
    return res

# Loop grid
for sl in GRID_SL:
    for tp in GRID_TP:
        if tp <= sl: continue
        try:
            row = run_config(sl, tp)
            summary_rows.append(row)
        except Exception as e:
            print(f"Config SL={sl},TP={tp} failed: {e}")

# Save overall grid summary
summary_df = pd.DataFrame(summary_rows).sort_values(by="Sharpe", ascending=False).reset_index(drop=True)
summary_df.to_csv(OUTPUT_ROOT / "grid_summary.csv", index=False)
print("\nGrid done. Summary saved:", str(OUTPUT_ROOT / "grid_summary.csv"))

Tickers: 390 | Clusters in-use: 4

=== VAL_SL1_TP2 ===

=== VAL_SL1_TP4 ===

=== VAL_SL1_TP6 ===

=== VAL_SL1_TP10 ===

=== VAL_SL2_TP4 ===

=== VAL_SL2_TP6 ===

=== VAL_SL2_TP10 ===

=== VAL_SL3_TP4 ===

=== VAL_SL3_TP6 ===

=== VAL_SL3_TP10 ===

=== VAL_SL5_TP6 ===

=== VAL_SL5_TP10 ===

Grid done. Summary saved: backtest_grid_val\grid_summary.csv


**BLOCK 10C:chọnconfig**

In [94]:
# Block 10C — Backtest Test Set với SL/TP (min holding days = 2, fix NaN + clean log)
import os, pandas as pd, numpy as np
from pathlib import Path

SIG_DIR   = Path("./signals/")
BT_DIR    = Path("./backtest_test/")
os.makedirs(BT_DIR, exist_ok=True)

SIG_FILE  = SIG_DIR / "a3c_signals_infer.csv"   
DF_BACK   = Path("./backtest_ddpg/df_backtest.csv")

SL, TP    = 0.01, 0.04   # chọn từ kết quả val
MIN_HOLD  = 2            
capital0  = 1_000_000

# ---------- Load ----------
if not SIG_FILE.exists():
    raise FileNotFoundError(f"Không tìm thấy file signals TEST: {SIG_FILE}")
if not DF_BACK.exists():
    raise FileNotFoundError(f"Không tìm thấy df_backtest: {DF_BACK}")

signals = pd.read_csv(SIG_FILE, parse_dates=["date"])
df_px   = pd.read_csv(DF_BACK, parse_dates=["date"])
df_px   = df_px.rename(columns={"date":"timestamp"}).sort_values(["timestamp","ticker"])

# pivot giá
px_open  = df_px.pivot(index="timestamp", columns="ticker", values="open")
px_high  = df_px.pivot(index="timestamp", columns="ticker", values="high")
px_low   = df_px.pivot(index="timestamp", columns="ticker", values="low")
px_close = df_px.pivot(index="timestamp", columns="ticker", values="close")

dates   = sorted(list(set(signals["date"]) & set(px_close.index)))
tickers = sorted(signals["ticker"].unique())
print(f"Dates test: {dates[0]} → {dates[-1]} | {len(dates)} ngày | {len(tickers)} mã")

# ---------- Backtest ----------
capital   = capital0
positions = {}  
equity_curve = []
trade_log = []

for dt in dates:
    sig_today = signals[signals["date"]==dt].set_index("ticker")["signal"].to_dict()
    cap_avail = capital

    # update positions
    closed_today = []
    for tkr, pos in list(positions.items()):
        if tkr not in px_close.columns: 
            continue
        o,h,l,c = px_open.at[dt,tkr], px_high.at[dt,tkr], px_low.at[dt,tkr], px_close.at[dt,tkr]
        if any(pd.isna([o,h,l,c])): 
            continue

        pos["holding_days"] += 1
        exit_flag = False; exit_price = None; reason = None

        if pos["holding_days"] >= MIN_HOLD:
            if pos["side"]==1:  # long
                if l <= pos["entry"]*(1-SL):
                    exit_flag=True; exit_price=pos["entry"]*(1-SL); reason="SL"
                elif h >= pos["entry"]*(1+TP):
                    exit_flag=True; exit_price=pos["entry"]*(1+TP); reason="TP"
            elif pos["side"]==-1: # short
                if h >= pos["entry"]*(1+SL):
                    exit_flag=True; exit_price=pos["entry"]*(1+SL); reason="SL"
                elif l <= pos["entry"]*(1-TP):
                    exit_flag=True; exit_price=pos["entry"]*(1-TP); reason="TP"

        if exit_flag:
            pnl = (exit_price - pos["entry"]) / pos["entry"] * pos["side"] * pos["size"]
            capital += pnl
            trade_log.append({
                "ticker": tkr, "side": pos["side"],
                "date_entry": pos["entry_date"], "entry": pos["entry"],
                "date_exit": dt, "exit": exit_price,
                "size": pos["size"], "holding_days": pos["holding_days"],
                "pnl": pnl, "reason": reason
            })
            closed_today.append(tkr)

    for t in closed_today: 
        del positions[t]

    # mở vị thế mới
    for tkr, sig in sig_today.items():
        if sig==0 or tkr in positions: 
            continue
        if tkr not in px_open.columns: 
            continue
        entry_price = px_open.at[dt,tkr]
        if pd.isna(entry_price): 
            continue
        size = cap_avail/len(sig_today) if len(sig_today)>0 else 0
        positions[tkr] = {
            "side": sig, "entry": entry_price, "entry_date": dt,
            "size": size, "holding_days": 0
        }
        trade_log.append({
            "ticker": tkr, "side": sig,
            "date_entry": dt, "entry": entry_price,
            "size": size, "holding_days":0,
            "pnl": 0.0, "reason":"entry"
        })

    # mark-to-market equity
    eq_val = capital
    for t,pos in positions.items():
        if t in px_close.columns:
            px_val = px_close.at[dt,t]
            if pd.notna(px_val):
                eq_val += (px_val - pos["entry"]) / pos["entry"] * pos["side"] * pos["size"]
    if pd.isna(eq_val):
        eq_val = capital
    equity_curve.append({"date": dt, "equity": eq_val})

# ---------- Save ----------
equity_df = pd.DataFrame(equity_curve).set_index("date")
equity_df.to_csv(BT_DIR/"equity_curve.csv")

pd.DataFrame(trade_log).to_csv(BT_DIR/"trade_log.csv", index=False)

# extra files cho Block 11a-Test
equity_df.to_csv(BT_DIR/"portfolio_value.csv")
equity_df["returns"] = equity_df["equity"].pct_change().fillna(0.0)
equity_df[["returns"]].to_csv(BT_DIR/"daily_returns.csv")

print(f"✅ Block 10C hoàn tất. Saved to {BT_DIR}")

Dates test: 2025-04-10 00:00:00 → 2025-10-02 00:00:00 | 121 ngày | 391 mã
✅ Block 10C hoàn tất. Saved to backtest_test


In [6]:
# Block 10C — Backtest Test Set với SL/TP (Rebalance/Strict option + Fee)
import os, pandas as pd, numpy as np
from pathlib import Path

SIG_DIR   = Path("./signals/")
BT_DIR    = Path("./backtest_test/")
os.makedirs(BT_DIR, exist_ok=True)

SIG_FILE  = SIG_DIR / "a3c_signals_infer.csv"   
DF_BACK   = Path("./backtest_ddpg/df_backtest.csv")

# --- Params ---
SL, TP     = 0.01, 0.04   # stoploss & takeprofit
MIN_HOLD   = 2            
capital0   = 1_000_000
FEE_RATE   = 0.001        # 0.1% phí giao dịch
REBALANCE  = False        # True = chia lại vốn mỗi ngày, False = strict free-capital

# ---------- Load ----------
signals = pd.read_csv(SIG_FILE, parse_dates=["date"])
df_px   = pd.read_csv(DF_BACK, parse_dates=["date"])
df_px   = df_px.rename(columns={"date":"timestamp"}).sort_values(["timestamp","ticker"])

px_open  = df_px.pivot(index="timestamp", columns="ticker", values="open")
px_high  = df_px.pivot(index="timestamp", columns="ticker", values="high")
px_low   = df_px.pivot(index="timestamp", columns="ticker", values="low")
px_close = df_px.pivot(index="timestamp", columns="ticker", values="close")

dates   = sorted(list(set(signals["date"]) & set(px_close.index)))
tickers = sorted(signals["ticker"].unique())
print(f"Dates test: {dates[0]} → {dates[-1]} | {len(dates)} ngày | {len(tickers)} mã")

# ---------- Backtest ----------
capital   = capital0
positions = {}  
equity_curve = []
trade_log = []

for dt in dates:
    sig_today = signals[signals["date"]==dt].set_index("ticker")["signal"].to_dict()

    # ---------- Update positions ----------
    closed_today = []
    for tkr, pos in list(positions.items()):
        if tkr not in px_close.columns: 
            continue
        o,h,l,c = px_open.at[dt,tkr], px_high.at[dt,tkr], px_low.at[dt,tkr], px_close.at[dt,tkr]
        if any(pd.isna([o,h,l,c])): 
            continue

        pos["holding_days"] += 1
        exit_flag = False; exit_price = None; reason = None

        if pos["holding_days"] >= MIN_HOLD:
            if pos["side"]==1:  # long
                if l <= pos["entry"]*(1-SL):
                    exit_flag=True; exit_price=pos["entry"]*(1-SL); reason="SL"
                elif h >= pos["entry"]*(1+TP):
                    exit_flag=True; exit_price=pos["entry"]*(1+TP); reason="TP"
            elif pos["side"]==-1: # short
                if h >= pos["entry"]*(1+SL):
                    exit_flag=True; exit_price=pos["entry"]*(1+SL); reason="SL"
                elif l <= pos["entry"]*(1-TP):
                    exit_flag=True; exit_price=pos["entry"]*(1-TP); reason="TP"

        if exit_flag:
            # pnl raw
            pnl_raw = (exit_price - pos["entry"]) / pos["entry"] * pos["side"] * pos["size"]
            # phí
            fee_open  = pos["size"] * FEE_RATE
            fee_close = (pos["size"] * exit_price / pos["entry"]) * FEE_RATE
            pnl_after = pnl_raw - fee_open - fee_close
            capital += pnl_after

            trade_log.append({
                "ticker": tkr, "side": pos["side"],
                "date_entry": pos["entry_date"], "entry": pos["entry"],
                "date_exit": dt, "exit": exit_price,
                "size": pos["size"], "holding_days": pos["holding_days"],
                "pnl_raw": pnl_raw, "pnl_after_fee": pnl_after, "reason": reason
            })
            closed_today.append(tkr)

    for t in closed_today: 
        del positions[t]

    # ---------- Mở vị thế mới ----------
    if REBALANCE:
        # phân bổ lại vốn đều cho toàn bộ positions + lệnh mới
        n_new = sum([1 for tkr,sig in sig_today.items() if sig!=0 and tkr not in positions])
        n_total = len(positions) + n_new
        if n_total > 0:
            alloc = capital / n_total
            # cập nhật lại size các pos cũ
            for tkr,pos in positions.items():
                pos["size"] = alloc
            # mở pos mới
            for tkr,sig in sig_today.items():
                if sig==0 or tkr in positions: continue
                entry_price = px_open.at[dt,tkr]
                if pd.isna(entry_price): continue
                positions[tkr] = {
                    "side": sig, "entry": entry_price, "entry_date": dt,
                    "size": alloc, "holding_days": 0
                }
                fee_open = alloc * FEE_RATE
                trade_log.append({
                    "ticker": tkr, "side": sig,
                    "date_entry": dt, "entry": entry_price,
                    "size": alloc, "holding_days":0,
                    "pnl_raw": 0.0, "pnl_after_fee": -fee_open,
                    "reason":"entry"
                })
                capital -= fee_open
    else:
        # strict free-capital
        free_cap = capital - sum([p["size"] for p in positions.values()])
        new_sigs = [tkr for tkr,sig in sig_today.items() if sig!=0 and tkr not in positions]
        if free_cap > 0 and len(new_sigs)>0:
            alloc = free_cap / len(new_sigs)
            for tkr in new_sigs:
                entry_price = px_open.at[dt,tkr]
                if pd.isna(entry_price): continue
                positions[tkr] = {
                    "side": sig_today[tkr], "entry": entry_price, "entry_date": dt,
                    "size": alloc, "holding_days": 0
                }
                fee_open = alloc * FEE_RATE
                trade_log.append({
                    "ticker": tkr, "side": sig_today[tkr],
                    "date_entry": dt, "entry": entry_price,
                    "size": alloc, "holding_days":0,
                    "pnl_raw": 0.0, "pnl_after_fee": -fee_open,
                    "reason":"entry"
                })
                capital -= fee_open

    # ---------- mark-to-market equity ----------
    eq_val = capital
    for t,pos in positions.items():
        if t in px_close.columns:
            px_val = px_close.at[dt,t]
            if pd.notna(px_val):
                eq_val += (px_val - pos["entry"]) / pos["entry"] * pos["side"] * pos["size"]
    equity_curve.append({"date": dt, "equity": eq_val})

# ---------- Save ----------
equity_df = pd.DataFrame(equity_curve).set_index("date")
equity_df.to_csv(BT_DIR/"equity_curve.csv")
pd.DataFrame(trade_log).to_csv(BT_DIR/"trade_log.csv", index=False)

equity_df.to_csv(BT_DIR/"portfolio_value.csv")
equity_df["returns"] = equity_df["equity"].pct_change().fillna(0.0)
equity_df[["returns"]].to_csv(BT_DIR/"daily_returns.csv")

print(f"✅ Block 10C hoàn tất. Mode={'Rebalance' if REBALANCE else 'Strict'}, Fee={FEE_RATE*100:.2f}%")

Dates test: 2025-02-06 00:00:00 → 2025-10-02 00:00:00 | 165 ngày | 391 mã
✅ Block 10C hoàn tất. Mode=Strict, Fee=0.10%


In [33]:
# Block 10C — Backtest Test Set với SL/TP (Strict free-capital + Top-k signals + Fee)
import os, pandas as pd, numpy as np
from pathlib import Path

SIG_DIR   = Path("./signals/")
BT_DIR    = Path("./backtest_test/")
os.makedirs(BT_DIR, exist_ok=True)

SIG_FILE  = SIG_DIR / "a3c_signals_infer.csv"   
DF_BACK   = Path("./backtest_ddpg/df_backtest.csv")

# --- Params ---
SL, TP       = 0.01, 0.04   # stoploss & takeprofit (cố định)
MIN_HOLD     = 2            
capital0     = 1_000_000
FEE_RATE     = 0.001        # 0.1% phí giao dịch
MAX_POSITIONS= 100           # tổng số mã tối đa giữ trong danh mục
TOPK_DAILY   = 30            # số tín hiệu tối đa mở mới mỗi ngày

# ---------- Load ----------
signals = pd.read_csv(SIG_FILE, parse_dates=["date"])
df_px   = pd.read_csv(DF_BACK, parse_dates=["date"])
df_px   = df_px.rename(columns={"date":"timestamp"}).sort_values(["timestamp","ticker"])

px_open  = df_px.pivot(index="timestamp", columns="ticker", values="open")
px_high  = df_px.pivot(index="timestamp", columns="ticker", values="high")
px_low   = df_px.pivot(index="timestamp", columns="ticker", values="low")
px_close = df_px.pivot(index="timestamp", columns="ticker", values="close")

dates   = sorted(list(set(signals["date"]) & set(px_close.index)))
tickers = sorted(signals["ticker"].unique())
print(f"Dates test: {dates[0]} → {dates[-1]} | {len(dates)} ngày | {len(tickers)} mã")

# ---------- Backtest ----------
capital   = capital0
positions = {}  
equity_curve = []
trade_log = []

for dt in dates:
    sig_today = signals[signals["date"]==dt].set_index("ticker")["signal"].to_dict()

    # ---------- Update positions ----------
    closed_today = []
    for tkr, pos in list(positions.items()):
        if tkr not in px_close.columns: 
            continue
        o,h,l,c = px_open.at[dt,tkr], px_high.at[dt,tkr], px_low.at[dt,tkr], px_close.at[dt,tkr]
        if any(pd.isna([o,h,l,c])): 
            continue

        pos["holding_days"] += 1
        exit_flag = False; exit_price = None; reason = None

        if pos["holding_days"] >= MIN_HOLD:
            if pos["side"]==1:  # long
                if l <= pos["entry"]*(1-SL):
                    exit_flag=True; exit_price=pos["entry"]*(1-SL); reason="SL"
                elif h >= pos["entry"]*(1+TP):
                    exit_flag=True; exit_price=pos["entry"]*(1+TP); reason="TP"
            elif pos["side"]==-1: # short
                if h >= pos["entry"]*(1+SL):
                    exit_flag=True; exit_price=pos["entry"]*(1+SL); reason="SL"
                elif l <= pos["entry"]*(1-TP):
                    exit_flag=True; exit_price=pos["entry"]*(1-TP); reason="TP"

        if exit_flag:
            pnl_raw = (exit_price - pos["entry"]) / pos["entry"] * pos["side"] * pos["size"]
            fee_open  = pos["size"] * FEE_RATE
            fee_close = (pos["size"] * exit_price / pos["entry"]) * FEE_RATE
            pnl_after = pnl_raw - fee_open - fee_close
            capital += pnl_after

            trade_log.append({
                "ticker": tkr, "side": pos["side"],
                "date_entry": pos["entry_date"], "entry": pos["entry"],
                "date_exit": dt, "exit": exit_price,
                "size": pos["size"], "holding_days": pos["holding_days"],
                "pnl_raw": pnl_raw, "pnl_after_fee": pnl_after, "reason": reason
            })
            closed_today.append(tkr)

    for t in closed_today: 
        del positions[t]

    # ---------- Mở vị thế mới (giới hạn Top-k + Max positions) ----------
    if len(positions) < MAX_POSITIONS:
        free_cap = capital - sum([p["size"] for p in positions.values()])
        new_sigs = [tkr for tkr,sig in sig_today.items() if sig!=0 and tkr not in positions]
        if len(new_sigs) > 0 and free_cap > 0:
            # chỉ lấy tối đa TOPK_DAILY
            new_sigs = new_sigs[:TOPK_DAILY]
            # nếu quá MAX_POSITIONS thì cắt bớt
            slots_avail = MAX_POSITIONS - len(positions)
            new_sigs = new_sigs[:slots_avail]
            # phân bổ vốn
            alloc = free_cap / len(new_sigs) if len(new_sigs)>0 else 0
            for tkr in new_sigs:
                entry_price = px_open.at[dt,tkr]
                if pd.isna(entry_price): continue
                positions[tkr] = {
                    "side": sig_today[tkr], "entry": entry_price, "entry_date": dt,
                    "size": alloc, "holding_days": 0
                }
                fee_open = alloc * FEE_RATE
                trade_log.append({
                    "ticker": tkr, "side": sig_today[tkr],
                    "date_entry": dt, "entry": entry_price,
                    "size": alloc, "holding_days":0,
                    "pnl_raw": 0.0, "pnl_after_fee": -fee_open,
                    "reason":"entry"
                })
                capital -= fee_open

    # ---------- mark-to-market equity ----------
    eq_val = capital
    for t,pos in positions.items():
        if t in px_close.columns:
            px_val = px_close.at[dt,t]
            if pd.notna(px_val):
                eq_val += (px_val - pos["entry"]) / pos["entry"] * pos["side"] * pos["size"]
    equity_curve.append({"date": dt, "equity": eq_val})

# ---------- Save ----------
equity_df = pd.DataFrame(equity_curve).set_index("date")
equity_df.to_csv(BT_DIR/"equity_curve.csv")
pd.DataFrame(trade_log).to_csv(BT_DIR/"trade_log.csv", index=False)

equity_df.to_csv(BT_DIR/"portfolio_value.csv")
equity_df["returns"] = equity_df["equity"].pct_change().fillna(0.0)
equity_df[["returns"]].to_csv(BT_DIR/"daily_returns.csv")

print(f"✅ Block 10C hoàn tất. Mode=Strict, Fee={FEE_RATE*100:.2f}%, MaxPos={MAX_POSITIONS}, TopK={TOPK_DAILY}")

Dates test: 2025-02-06 00:00:00 → 2025-10-02 00:00:00 | 165 ngày | 391 mã
✅ Block 10C hoàn tất. Mode=Strict, Fee=0.10%, MaxPos=100, TopK=30


**Block 11: Thống kê kết quả và vẽ biểu đồ**

In [34]:
# Block 11a-Test — Hiệu suất & Stress Test (Test set, SL/TP cố định)
import os, numpy as np, pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import statsmodels.api as sm

OUTPUT_DIR   = "./backtest_test/"
BEST_DIR     = "./backtest_best_test/"
os.makedirs(BEST_DIR, exist_ok=True)

# ---------- Helpers ----------
def max_drawdown(series: pd.Series) -> float:
    peak = series.cummax()
    dd = (series / peak) - 1.0
    return float(dd.min())

def compute_stats(port_val: pd.Series, bench_val: pd.Series | None = None):
    port_ret = port_val.pct_change().fillna(0.0)
    stats = {
        "Ngày bắt đầu": port_val.index.min().strftime("%Y-%m-%d"),
        "Ngày kết thúc": port_val.index.max().strftime("%Y-%m-%d"),
        "Giá trị cuối": float(port_val.iloc[-1]),
        "ROI (%)": float((port_val.iloc[-1] / port_val.iloc[0] - 1.0) * 100),
        "Biến động (năm, %)": float(port_ret.std() * np.sqrt(252) * 100),
        "Sharpe": float((port_ret.mean() / port_ret.std()) * np.sqrt(252)) if port_ret.std() > 0 else 0,
        "Sortino": float((port_ret.mean() / port_ret[port_ret < 0].std()) * np.sqrt(252)) if port_ret[port_ret < 0].std() > 0 else 0,
        "MaxDrawdown (%)": float(max_drawdown(port_val) * 100),
        "Tỷ lệ phiên thắng (%)": float((port_ret > 0).mean() * 100),
        "Số ngày": int(len(port_ret))
    }
    if bench_val is not None and len(bench_val) > 1:
        stats.update({
            "Giá trị cuối (VNINDEX)": float(bench_val.iloc[-1]),
            "ROI VNINDEX (%)": float((bench_val.iloc[-1] / bench_val.iloc[0] - 1.0) * 100),
            "Chênh lệch so với VNINDEX (pp)": stats["ROI (%)"] - ((bench_val.iloc[-1] / bench_val.iloc[0] - 1.0) * 100)
        })
    return stats

def plot_equity(port_val, bench_val, title, path):
    plt.figure(figsize=(10,6))
    plt.plot(port_val, label="Chiến lược", linewidth=1.6)
    if bench_val is not None:
        plt.plot(bench_val, label="VNINDEX", linewidth=1.2)
    plt.title(title); plt.legend(); plt.grid(True, alpha=0.3)
    plt.savefig(path, dpi=150); plt.close()

# ---------- 1. Load dữ liệu ----------
eq_path = os.path.join(OUTPUT_DIR, "equity_curve.csv")
if not os.path.exists(eq_path):
    raise FileNotFoundError("Chưa có equity_curve.csv từ Block 10C")

port_val = pd.read_csv(eq_path, index_col=0, parse_dates=True)["equity"].sort_index()
daily_returns = port_val.pct_change().fillna(0.0)

# benchmark từ df_backtest
df_backtest_path = "./backtest_ddpg/df_backtest.csv"
bench_val, bench_ret = None, None
if os.path.exists(df_backtest_path):
    df_backtest = pd.read_csv(df_backtest_path, parse_dates=["date"])
    df_backtest = df_backtest.groupby(["date","ticker"],as_index=False).agg({"close":"last"})
    px = df_backtest.pivot(index="date", columns="ticker", values="close").sort_index()
    if "VNINDEX" in px.columns:
        bench_val = px["VNINDEX"].reindex(port_val.index).ffill().bfill()
        bench_ret = bench_val.pct_change().fillna(0.0)

# ---------- 2. Compute Stats ----------
stats_test = compute_stats(port_val, bench_val)

# ---------- 3. Plot cơ bản ----------
plot_equity(port_val, bench_val, "Equity Curve (Test set)", os.path.join(BEST_DIR,"equity.png"))

plt.figure(figsize=(9,5))
plt.hist(daily_returns.dropna(), bins=50, alpha=0.6, label="Chiến lược")
if bench_ret is not None:
    plt.hist(bench_ret.dropna(), bins=50, alpha=0.6, label="VNINDEX")
plt.title("Histogram lợi nhuận ngày"); plt.legend(); plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(BEST_DIR,"hist.png")); plt.close()

# ---------- 4. Stress Test (ví dụ Trump 46%) ----------
stress_start, stress_end = pd.Timestamp("2025-03-26"), pd.Timestamp("2025-04-15")
sub_port = port_val.loc[stress_start:stress_end]
sub_bench = bench_val.loc[stress_start:stress_end] if bench_val is not None else None
stats_stress = {}
if len(sub_port) > 1:
    stats_stress = compute_stats(sub_port, sub_bench)
    plot_equity(sub_port, sub_bench, "Stress Test (Test set)", os.path.join(BEST_DIR,"equity_stress.png"))

# ---------- 5. Save ----------
all_stats={"TestSet":stats_test}
if stats_stress: all_stats["StressTest"]=stats_stress
pd.DataFrame(all_stats).T.to_csv(os.path.join(BEST_DIR,"stats.csv"))

print("\n📊 Kết quả Test Set:"); print(pd.Series(stats_test))
if stats_stress: 
    print("\n📊 Stress Test:"); print(pd.Series(stats_stress))
print(f"\n✅ Block 11a-Test hoàn tất. Lưu kết quả tại {BEST_DIR}")


📊 Kết quả Test Set:
Ngày bắt đầu               2025-02-06
Ngày kết thúc              2025-10-02
Giá trị cuối             1169993.2623
ROI (%)                     16.929174
Biến động (năm, %)          14.988755
Sharpe                       1.668602
Sortino                      2.103304
MaxDrawdown (%)             -7.433818
Tỷ lệ phiên thắng (%)            60.0
Số ngày                           165
dtype: object

📊 Stress Test:
Ngày bắt đầu                 2025-03-26
Ngày kết thúc                2025-04-15
Giá trị cuối             1008343.239927
ROI (%)                       -0.302326
Biến động (năm, %)            33.888843
Sharpe                        -0.005699
Sortino                       -0.008435
MaxDrawdown (%)               -5.814073
Tỷ lệ phiên thắng (%)         35.714286
Số ngày                              14
dtype: object

✅ Block 11a-Test hoàn tất. Lưu kết quả tại ./backtest_best_test/


In [35]:
# Block 11b-Test — Full performance & diagnostics (Strategy vs VNINDEX benchmark)
import os, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from FiinQuantX import FiinSession
import statsmodels.api as sm

warnings.filterwarnings("ignore")
plt.rcParams["figure.dpi"] = 150

# ---------- Paths ----------
BACKTEST_TEST_DIR = Path("./backtest_test")
OUT_DIR           = Path("./backtest_best_test")
OUT_DIR.mkdir(parents=True, exist_ok=True)

EQ_PATH = BACKTEST_TEST_DIR / "equity_curve.csv"
if not EQ_PATH.exists():
    raise FileNotFoundError(f"Không tìm thấy equity_curve.csv tại {EQ_PATH}. Chạy Block 10C trước.")

# ---------- Load portfolio equity ----------
eq_df = pd.read_csv(EQ_PATH, parse_dates=[0], index_col=0).sort_index()
port_val = eq_df["equity"] if "equity" in eq_df.columns else eq_df.iloc[:,0]
port_val = port_val.astype(float).replace([np.inf, -np.inf], np.nan).ffill().bfill()

# ---------- Fetch VNINDEX benchmark ----------
print("📈 Fetching VNINDEX from FiinQuantX...")
client = FiinSession(username="DSTC_18@fiinquant.vn", password="Fiinquant0606").login()
bench = client.Fetch_Trading_Data(
    realtime=False, tickers="VNINDEX", fields=['close'],
    adjusted=True, by="1d", from_date=str(port_val.index.min().date())
).get_data()
bench["date"] = pd.to_datetime(bench["timestamp"])
bench = bench.set_index("date")["close"].sort_index().reindex(port_val.index).ffill().bfill()

INIT_CAP = float(port_val.iloc[0])
bench_equity = (1.0 + bench.pct_change().fillna(0.0)).cumprod() * INIT_CAP

# ---------- Stats helper ----------
def max_drawdown(series):
    peak = series.cummax()
    return float(((series/peak)-1).min())

def compute_stats(port_series, bench_series=None):
    r = port_series.pct_change().dropna()
    roi = (port_series.iloc[-1]/port_series.iloc[0]-1.0)
    vol = r.std()*np.sqrt(252)
    sharpe = (r.mean()/r.std())*np.sqrt(252) if r.std()>0 else 0.0
    sortino = (r.mean()/r[r<0].std())*np.sqrt(252) if r[r<0].std()>0 else 0.0
    mdd = max_drawdown(port_series)
    win = (r>0).mean()
    stats = {
        "Start": str(port_series.index.min().date()),
        "End": str(port_series.index.max().date()),
        "Initial": float(port_series.iloc[0]),
        "Final": float(port_series.iloc[-1]),
        "ROI": roi, "Vol(ann)": vol, "Sharpe": sharpe,
        "Sortino": sortino, "MaxDD": mdd, "WinRate": win
    }
    if bench_series is not None:
        broi = (bench_series.iloc[-1]/bench_series.iloc[0]-1.0)
        stats.update({"BenchFinal": float(bench_series.iloc[-1]), "BenchROI": broi, "ExcessROI": roi-broi})
    return stats

stats = compute_stats(port_val, bench_equity)
pd.Series(stats).to_csv(OUT_DIR/"stats_test.csv")

# ---------- Charts ----------
# Equity curve
plt.figure(figsize=(10,6))
plt.plot(port_val, label="Strategy", linewidth=2)
plt.plot(bench_equity, label="VNINDEX", linewidth=1.5)
plt.title("Equity Curve — Strategy vs VNINDEX")
plt.legend(); plt.grid(alpha=0.3)
plt.savefig(OUT_DIR/"equity_vs_benchmark.png"); plt.close()

# Drawdown
dd = (port_val/port_val.cummax()-1)
plt.figure(figsize=(10,5))
plt.plot(dd, label="Drawdown")
plt.fill_between(dd.index, dd, 0, alpha=0.3)
plt.title("Drawdown Curve")
plt.grid(alpha=0.3)
plt.savefig(OUT_DIR/"drawdown.png"); plt.close()

# Histogram
port_ret = port_val.pct_change().dropna()
bench_ret = bench_equity.pct_change().dropna()
plt.figure(figsize=(9,5))
plt.hist(port_ret, bins=50, alpha=0.6, label="Strategy")
plt.hist(bench_ret, bins=50, alpha=0.4, label="VNINDEX")
plt.title("Histogram daily returns")
plt.legend(); plt.grid(alpha=0.3)
plt.savefig(OUT_DIR/"hist.png"); plt.close()

# Monthly heatmap
monthly = port_ret.add(1).resample("M").prod().sub(1)
heat = monthly.to_frame("ret")
heat["Year"] = heat.index.year; heat["Month"] = heat.index.month
hm = heat.pivot(index="Year", columns="Month", values="ret").fillna(0)
plt.figure(figsize=(10,6))
sns.heatmap(hm, annot=True, fmt=".1%", center=0, cmap="RdYlGn")
plt.title("Monthly Return Heatmap")
plt.savefig(OUT_DIR/"heatmap.png"); plt.close()

# Rolling Sharpe (60d)
roll_sharpe = port_ret.rolling(60).mean() / port_ret.rolling(60).std() * np.sqrt(252)
plt.figure(figsize=(10,5))
plt.plot(roll_sharpe, label="Rolling Sharpe (60d)")
plt.axhline(0, color="black", linewidth=1)
plt.title("Rolling Sharpe Ratio (60d window)")
plt.grid(alpha=0.3); plt.legend()
plt.savefig(OUT_DIR/"rolling_sharpe.png"); plt.close()

# Rolling Beta (60d OLS vs VNINDEX)
roll_beta = []
for i in range(60, len(port_ret)):
    y = port_ret.iloc[i-60:i].values
    x = bench_ret.iloc[i-60:i].values
    if np.std(x)>0:
        model = sm.OLS(y, sm.add_constant(x)).fit()
        roll_beta.append((port_ret.index[i], model.params[1]))
roll_beta = pd.DataFrame(roll_beta, columns=["date","beta"]).set_index("date")
plt.figure(figsize=(10,5))
plt.plot(roll_beta["beta"], label="Rolling Beta (60d)")
plt.axhline(1, color="red", linestyle="--", linewidth=1)
plt.title("Rolling Beta vs VNINDEX")
plt.grid(alpha=0.3); plt.legend()
plt.savefig(OUT_DIR/"rolling_beta.png"); plt.close()

# ---------- Stress Test (same window as 11a) ----------
stress_start, stress_end = pd.Timestamp("2025-03-26"), pd.Timestamp("2025-04-15")
sub_port = port_val.loc[stress_start:stress_end]
sub_bench = bench_equity.loc[stress_start:stress_end]

if len(sub_port) > 1:
    stress_stats = compute_stats(sub_port, sub_bench)
    pd.Series(stress_stats).to_csv(OUT_DIR/"stats_stress.csv")
    plt.figure(figsize=(10,6))
    plt.plot(sub_port, label="Strategy", linewidth=2)
    plt.plot(sub_bench, label="VNINDEX", linewidth=1.5)
    plt.title("Stress Test Equity Curve")
    plt.legend(); plt.grid(alpha=0.3)
    plt.savefig(OUT_DIR/"equity_stress.png"); plt.close()
    print("📊 Stress Test Stats saved to", OUT_DIR/"stats_stress.csv")
    

print("✅ Block 11b-Test FULL hoàn tất. Stats + Charts lưu tại:", OUT_DIR)

📈 Fetching VNINDEX from FiinQuantX...
Fetching data, it may take a while. Please wait...
📊 Stress Test Stats saved to backtest_best_test\stats_stress.csv
✅ Block 11b-Test FULL hoàn tất. Stats + Charts lưu tại: backtest_best_test


**Block 12A: gửi tín hiệu lên telegram với dữ liệu quá khứ**

`Lưu ý` tuy quá khứ nhưng nhóm không để bị nhìn trước tương lai 

In [39]:
# Block 12a — Gửi báo cáo Telegram (Summary + Chart + CSV chi tiết tùy chọn)
import os, json, requests, time
import pandas as pd
import matplotlib.pyplot as plt

# ==== Đường dẫn ====
BT_DIR = "./backtest_test/"
EQ_PATH = os.path.join(BT_DIR, "equity_curve.csv")
LOG_PATH = os.path.join(BT_DIR, "trade_log.csv")

# ==== Load config ====
with open("config.json","r") as f:
    cfg = json.load(f)

TG_TOKEN     = cfg["telegram"]["bot_token"]
TG_CHAT_ID   = cfg["telegram"]["chat_id"]
TG_THREAD_ID = cfg["telegram"]["message_thread_id"]

# ==== Load dữ liệu backtest ====
equity = pd.read_csv(EQ_PATH, parse_dates=["date"], index_col="date")["equity"]
trades = pd.read_csv(LOG_PATH, parse_dates=["date_entry","date_exit"])

def send_daily_report(report_date: str | pd.Timestamp, details: bool=False, sleep_sec:int=2):
    """
    Gửi báo cáo Telegram cho 1 ngày trong backtest.
    - Summary text
    - Equity chart
    - Tùy chọn gửi CSV chi tiết (opened, closed, open_positions)
    """
    report_date = pd.Timestamp(report_date).normalize()
    if report_date not in equity.index:
        print(f"⚠️ {report_date.date()} không có trong equity index")
        return

    # --- NAV ---
    nav_today = equity.loc[:report_date].iloc[-1]

    # --- Lọc giao dịch ---
    closed_today = trades[trades["date_exit"].dt.normalize() == report_date]
    opened_today = trades[(trades["date_entry"].dt.normalize() == report_date) & (trades["reason"]=="entry")]

    # --- Danh mục còn mở đến hôm nay (fix) ---
    open_positions = trades[
        (trades["date_entry"].dt.normalize() <= report_date) &
        ((trades["date_exit"].isna()) | (trades["date_exit"].dt.normalize() > report_date))
    ].sort_values("date_entry").drop_duplicates(subset=["ticker"], keep="last")

    # --- % lãi/lỗ TB hôm nay ---
    avg_pnl = None
    if len(closed_today) > 0:
        pnl_pct = ((closed_today["exit"]/closed_today["entry"] - 1) * closed_today["side"]) * 100
        avg_pnl = pnl_pct.mean()

    # --- Danh sách ticker (giới hạn 5 để gọn) ---
    def list_names(df, label):
        if len(df) == 0:
            return f"{label}: Không có"
        names = df["ticker"].astype(str).tolist()
        if len(names) > 5:
            return f"{label}: {', '.join(names[:5])}... (+{len(names)-5} mã nữa)"
        return f"{label}: {', '.join(names)}"

    opened_txt = list_names(opened_today, "🟢 Mở")
    closed_txt = list_names(closed_today, "🔴 Đóng")

    # --- Compose message ---
    msg = f"""
📅 {report_date.date()}

💰 NAV: {nav_today:,.0f}

📊 Đang mở: {len(open_positions)} mã
{opened_txt}
{closed_txt}
{'' if avg_pnl is None else f'↳ Lãi/lỗ TB: {avg_pnl:.2f}%'}
""".strip()

    # --- Chart equity đến ngày đó ---
    chart_path = os.path.join(BT_DIR, f"equity_until_{report_date.date()}.png")
    plt.figure(figsize=(8,5))
    plt.plot(equity.loc[:report_date], label="Chiến lược")
    plt.title(f"Equity đến {report_date.date()}")
    plt.grid(True, alpha=0.3); plt.legend()
    plt.savefig(chart_path, dpi=150); plt.close()

    # --- Gửi text ---
    send_url = f"https://api.telegram.org/bot{TG_TOKEN}/sendMessage"
    requests.post(send_url, data={
        "chat_id": TG_CHAT_ID,
        "message_thread_id": TG_THREAD_ID,
        "text": msg
    })
    time.sleep(sleep_sec)

    # --- Gửi ảnh ---
    photo_url = f"https://api.telegram.org/bot{TG_TOKEN}/sendPhoto"
    with open(chart_path,"rb") as f:
        requests.post(photo_url, data={
            "chat_id": TG_CHAT_ID,
            "message_thread_id": TG_THREAD_ID,
            "caption": f"Equity Curve đến {report_date.date()}"
        }, files={"photo":f})
    time.sleep(sleep_sec)

    # --- Nếu bật details: gửi CSV chi tiết ---
    if details:
        csv_path = os.path.join(BT_DIR, f"trade_log_detail_{report_date.date()}.csv")
        detail_df = pd.concat([opened_today, closed_today, open_positions])
        detail_df.to_csv(csv_path, index=False)

        file_url = f"https://api.telegram.org/bot{TG_TOKEN}/sendDocument"
        with open(csv_path,"rb") as f:
            requests.post(file_url, data={
                "chat_id": TG_CHAT_ID,
                "message_thread_id": TG_THREAD_ID,
                "caption": f"Chi tiết giao dịch {report_date.date()}"
            }, files={"document":f})
        time.sleep(sleep_sec)

    print(f"✅ Đã gửi báo cáo Telegram cho ngày {report_date.date()} (details={details})")

In [40]:
# Replay báo cáo Telegram cho giai đoạn 26/3 → 15/4/2025
start_date, end_date = pd.Timestamp("2025-03-26"), pd.Timestamp("2025-04-15")

for dt in equity.loc[start_date:end_date].index:
    send_daily_report(dt, details=True, sleep_sec=2)

✅ Đã gửi báo cáo Telegram cho ngày 2025-03-26 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-03-27 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-03-28 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-03-31 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-01 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-02 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-03 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-04 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-08 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-09 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-10 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-11 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-14 (details=True)
✅ Đã gửi báo cáo Telegram cho ngày 2025-04-15 (details=True)
